<a href="https://colab.research.google.com/github/yjjangg/yjjang/blob/main/Ch2~Ch5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **2-1. pandas data structure**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import sys
os.listdir()

In [ ]:
cd drive

In [ ]:
os.listdir()


In [ ]:
cd MyDrive/Hands-On-Data-Analysis-with-Pandas-2nd-edition/

In [ ]:
cd ch_02

In [ ]:
import numpy as np

data = np.genfromtxt(
    'data/example_data.csv', delimiter=';', 
    names=True, dtype=None, encoding='UTF'
)
data

We can find the dimensions with the `shape` attribute:

In [ ]:
data[0]

We can find the data types with the `dtype` attribute:

In [ ]:
data.dtype

Each of the entries in the array is a row from the CSV file. NumPy arrays contain a single data type (unlike lists, which allow mixed types); this allows for fast, vectorized operations. When we read in the data, we got an array of `numpy.void` objects, which are created to store flexible types. This is because NumPy has to store several different data types per row: four strings, a float, and an integer. This means we can't take advantage of the performance improvements NumPy provides for single data type objects.

Say we want to find the maximum magnitude&mdash;we can use a **[list comprehension](https://www.python.org/dev/peps/pep-0202/)** to select the third index of each row, which is represented as a `numpy.void` object. This makes a list, meaning that we can take the maximum using the `max()` function:

In [ ]:
data

In [ ]:
max([row[3] for row in data])

In [ ]:
[row[3] for row in data]

In [ ]:
%%timeit
max([row[3] for row in data])

If we, instead, create a NumPy array for each column, this operation is much easier (and more efficient) to perform. We can use a **[dictionary comprehension](https://www.python.org/dev/peps/pep-0274/)** to make a dictionary where the keys are the column names and the values are NumPy arrays of the data:

In [ ]:
np.array([row[i] for row in data])

In [ ]:
array_dict = {
    col: np.array([row[i] for row in data])
    for i, col in enumerate(data.dtype.names)
}
array_dict

Grabbing the maximum magnitude is now simply a matter of selecting the `mag` key and calling the `max()` method. This is nearly twice as fast as the list comprehension implementation when dealing with just 5 entries, imagine how much worse the first attempt will perform on large data sets:

In [ ]:
array_dict['mag']

In [ ]:
%%timeit
array_dict['mag'].max()

However, this representation has other issues. Say we wanted to grab all the information for the earthquake with the maximum magnitude, how would we go about that? We would need to find the index of the maximum and then for each of the keys in the dictionary grab that index:

In [ ]:
array_dict.items()

In [ ]:
np.array([
    value[array_dict['mag'].argmax()] 
    for key, value in array_dict.items()
])

The result is now a NumPy array of strings (our numeric values were converted), and we are now in the format from earlier. Also, consider trying to sort the data by magnitude from smallest to largest. In the first representation, we would have to sort the rows by examining the 3rd index. With the second representation, we would have to determine the order for the indices from the `mag` column, and then sort all the other arrays with those same indices. Clearly, working with several NumPy arrays of different data types at once is a bit cumbersome. However, `pandas` builds on top of NumPy arrays to make this easier. Let's start our exploration of `pandas` with an overview of the data structures.

## `Series`
The `Series` class provides a data structure for arrays of a single type with some additional functionality.

In [ ]:
array_dict['place']

In [ ]:
import pandas as pd

place = pd.Series(array_dict['place'], name='place')
place

In [ ]:
place.index.values

In [ ]:
place.values

Here are some commonly used attributes with `Series` objects:

|Attribute | Returns |
| --- | --- |
| `name` | The name of the `Series` object |
| `dtype` | The data type of the `Series` object |
| `shape` | Dimensions of the `Series` object in a tuple of the form `(number of rows,)` |
| `index` | The `Index` object that is part of the `Series` object |
| `values` | The data in the `Series` object |

For the most part, `pandas` objects use NumPy arrays for their internal data representations. However, for some data types, `pandas` builds upon NumPy to create its own [arrays](https://pandas.pydata.org/pandas-docs/stable/reference/arrays.html). For this reason, depending on the data type, `values` can either be a `pandas.array` or `numpy.array` object. Therefore, if we need to ensure we get a specific type back, then it is recommended to use the `array` attribute or `to_numpy()` method, respectively, instead of `values`.

Now let's see some examples using these attributes.

### Getting the name of the series
The NumPy array held the name of the data in the `dtype` attribute; here, we can access it directly: 

In [ ]:
place.index

### Getting the data type
A `Series` object holds a single data type. Here it is `'O'` for object.

In [ ]:
place.dtype

### Getting the dimensions of the series
Just as with NumPy, we can use `shape` to get the dimensions as `(rows, columns)`. `Series` objects are a single column, so they only have values for the rows dimension. 

In [ ]:
place.shape

### Isolating the values from the series
This `Series` object is storing its values as a NumPy array:

In [ ]:
place.values

## `Index`
The addition of the `Index` class makes the `Series` class more powerful than a NumPy array. We can get the index from the `index` attribute of a `Series` object:

In [ ]:
place_index = place.index
place_index

As with `Series` objects, we can access the underlying data via the `values` attribute. Note that this `Index` object is also built on top of a NumPy array:

In [ ]:
place_index.values

Here are some commonly used attributes with `Index` objects:

|Attribute | Returns |
| --- | --- |
| `name` | The name of the `Index` object |
| `dtype` | The data type of the `Index` object |
| `shape` | Dimensions of the `Index` object |
| `values` | The data in the `Index` object |
| `is_unique` | Check if the `Index` object has all unique values |

We can check the type of the underlying data, just like with a `Series` object:

In [ ]:
place_index.dtype

Same for the dimensions:

In [ ]:
place_index.shape

We can check if the values are unique:

In [ ]:
place_index.is_unique

With NumPy we can perform arithmetic operations element-wise between arrays:

In [ ]:
np.array([1, 1, 1]) + np.array([-1, 0, 1])

Pandas supports this as well, and the index determines how element-wise operations are performed. With addition, only the matching indices are summed:

In [ ]:
numbers = np.linspace(0, 10, num=5) # makes numpy array([0, 2.5, 5, 7.5, 10])
x = pd.Series(numbers) # index is [0, 1, 2, 3, 4]
y = pd.Series(numbers, index=pd.Index([1, 2, 3, 4, 5]))
x + y

We aren't limited to the integer indices of list-like structures, and we can label our rows. The labels can be altered at any time and be things like dates or even another column. In chapter 3, we will discuss how to perform some operations on the index in order to change it. Then, in chapter 4, we will use the index for operations merging data and aggregating it.

## `DataFrame`
Having a `Series` object for each column is an improvement over the NumPy representation; however, we still have the same problem when wanting to sort based on a value or grab an entire row out. The `DataFrame` gives us a representation of a table formed from many `Series` objects that form the columns and a shared `Index` object that labels the rows. We can create a `DataFrame` object from either of the NumPy representations we were working with earlier (we could also make a `Series` object for each column, but there is no need to do so):

In [ ]:
array_dict

In [ ]:
df = pd.DataFrame(array_dict) 

# this will also work with the first representation
# df = pd.DataFrame(data)

df

We can check the type of the underlying data with `dtypes` (note that it is not `dtype` as with `Series` and `Index` objects since each column will have its own data type):

In [ ]:
df.dtypes

We can get the underlying data with the `values` attribute. Note that this looks very similar to our initial NumPy representation:

In [ ]:
df.values

We can isolate the columns with the `columns` attribute. Notice that the columns are actually an `Index` object just on a different axis (columns are the horizontal index while rows are the vertical index).

In [ ]:
df.index

Here are some commonly used attributes:

|Attribute | Returns |
| --- | --- |
| `dtypes` | The data types of each column |
| `shape` | Dimensions of the `DataFrame` object in a tuple of the form `(number of rows, number of columns)` |
| `index` | The `Index` object along the rows of the `DataFrame` object |
| `columns` | The name of the columns (as an `Index` object) |
| `values` | The data in the `DataFrame` object |
| `empty` | Check if the `DataFrame` object is empty |

The `Index` object along the rows of the dataframe can be accessed via the `index` attribute (just as with `Series` objects):

In [ ]:
df.index

As with both `Series` and `Index` objects, we can get the dimensions of the dataframe with the `shape` attribute. The result is of the form `(nrows, ncols)`. Our dataframe has 5 rows and 6 columns:

In [ ]:
df.shape

Note that we can also perform arithmetic on dataframes. Pandas will only perform the operation when both the index and column match. Here, we demonstrate addition. Since addition with strings means concatenation, `pandas` concatenated the string columns (`time`, `place`, `magType`, and `alert`) across dataframes. The numeric columns (`mag` and `tsunami`) were summed:

In [ ]:
df + df

# **2-2. creating dataframes**

In [ ]:
import datetime as dt
import numpy as np
import pandas as pd

## Creating a `Series` object

In [ ]:
np.random.seed(0) # set a seed for reproducibility
pd.Series(np.random.rand(5), name='random')

## Creating a `DataFrame` object from a `Series` object
Use the `to_frame()` method:

In [ ]:
pd.Series(np.linspace(0, 10, num=5)).to_frame()

## Creating a `DataFrame` from Python Data Structures
### From a dictionary of list-like structures
The dictionary values can be lists, NumPy arrays, etc. as long as they have length (generators don't have length so we can't use them here):

In [ ]:
np.random.seed(0) # set seed so result is reproducible
pd.DataFrame(
    {
        'random': np.random.rand(5),
        'text': ['hot', 'warm', 'cool', 'cold', None],
        'truth': [np.random.choice([True, False]) for _ in range(5)]
    }, 
    index=pd.date_range(
        end=dt.date(2019, 4, 21),
        freq='1D',
        periods=5, 
        name='date'
    )
)

### From a list of dictionaries

In [ ]:
pd.DataFrame([
    {'mag': 5.2, 'place': 'California'},
    {'mag': 1.2, 'place': 'Alaska'},
    {'mag': 0.2, 'place': 'California'},
])

### From a list of tuples

In [ ]:
list_of_tuples = [(n, n**2, n**3) for n in range(5)]
list_of_tuples

In [ ]:
pd.DataFrame(
    list_of_tuples, 
    columns=['n', 'n_squared', 'n_cubed']
)

### From a NumPy array

In [ ]:
df_1=pd.DataFrame(
    np.array([
        [0, 0, 0],
        [1, 1, 1],
        [2, 4, 8],
        [3, 9, 27],
        [4, 16, 64]
    ]), columns=['n', 'n_squared', 'n_cubed']
)
df_1

In [ ]:
df_1.mean

## Creating a `DataFrame` object from the contents of a CSV File

### Finding information on the file before reading it in
Before attempting to read in a file, we can use the command line to see important information about the file that may determine how we read it in. We can run command line code from Jupyter Notebooks (thanks to IPython) by using `!` before the code.

#### Number of lines (row count)
For example, we can find out how many lines are in the file by using the `wc` utility (word count) and counting lines in the file (`-l`). The file has 9,333 lines:

In [ ]:
!wc -l data/earthquakes.csv

**Windows users**: if the above doesn't work for you (depends on your setup), then use this instead:

```
!find /c /v "" data\earthquakes.csv
```



#### File size
We can find the file size by using `ls` to list the files in the `data` directory, and passing in the flags `-lh` to include the file size in human readable format. Then we use `grep` to find the file in question. Note that `|` passes the result of `ls` to `grep`. The `grep` utility is used for finding items that match patterns.

This tells us the file is 3.4 MB:

In [ ]:
!ls -lh data | grep earthquakes.csv

**Windows users**: if the above doesn't work for you (depends on your setup), then use this instead:

```
!dir data | findstr "earthquakes.csv"
```

We can even capture the result of a command and use it in our Python code:

In [ ]:
files = !ls -lh data
[file for file in files if 'earthquake' in file]

**Windows users**: if the above doesn't work for you (depends on your setup), then use this instead:

```
files = !dir data
[file for file in files if 'earthquake' in file]
```


#### Examining a few rows
We can use `head` to look at the top `n` rows of the file. With the `-n` flag, we can specify how many. This shows use that the first row of the file contains headers and that it is comma-separated (just because the file extension is `.csv` doesn't it contains comma-separated values):

In [ ]:
!head -n 2 data/earthquakes.csv

**Windows users**: if the above doesn't work for you (depends on your setup), then use this instead:

```
n = 2
with open('data/earthquakes.csv', 'r') as file:
    for _ in range(n):
        print(file.readline())
```


Just like `head` gives rows from the top, `tail` gives rows from the bottom. This can help us check that there is no extraneous data on the bottom of the field, like perhaps some metadata about the fields that actually isn't part of the dataset:

In [ ]:
!tail -n 1 data/earthquakes.csv

#### Column count
We can use `awk` to find the column count. This is a utility for pattern scanning and processing. The `-F` flag allows us to specify the delimiter (comma, in this case). Then we specify what to do for each record in the file. We choose to print `NF` which is a predefined variable whose value is the number of fields in the current record. Here, we say `exit` so that we print the number of fields (columns, here) in the first row of the file, then we stop. 

This tells us we have 26 data columns:

In [ ]:
!awk -F',' '{print NF; exit}' data/earthquakes.csv

**Windows users**: if the above or below don't work for you (depends on your setup), then use this instead:

```
import os

with open('data/earthquakes.csv', 'rb') as file:
    file.seek(-2, os.SEEK_END)
    while file.read(1) != b'\n':
        file.seek(-2, os.SEEK_CUR)
    print(file.readline().decode())
```


Since we know the 1st line of the file had headers, and the file is comma-separated, we can also count the columns by using `head` to get headers and parsing them in Python:

In [ ]:
headers = !head -n 1 data/earthquakes.csv
len(headers[0].split(','))

### Reading in the file
Our file is small in size, has headers in the first row, and is comma-separated, so we don't need to provide any additional arguments to read in the file with `pd.read_csv()`, but be sure to check the [documentation](http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) for possible arguments:

In [ ]:
df = pd.read_csv('data/earthquakes.csv')

In [ ]:
df.head()

Note that we can also pass in a URL. Let's read this same file from GitHub:

In [ ]:
df = pd.read_csv(
    'https://github.com/stefmolin/'
    'Hands-On-Data-Analysis-with-Pandas-2nd-edition'
    '/blob/master/ch_02/data/earthquakes.csv?raw=True'
)

Pandas is usually very good at figuring out which options to use based on the input data, so we often won't need to add arguments to the call; however, there are many options available should we need them, some of which include the following:

| Parameter | Purpose |
| --- | --- |
| `sep` | Specifies the delimiter |
| `header` | Row number where the column names are located; the default option has `pandas` infer whether they are present |
| `names` | List of column names to use as the header |
| `index_col` | Column to use as the index |
| `usecols` | Specifies which columns to read in |
| `dtype` | Specifies data types for the columns | 
| `converters` | Specifies functions for converting data in certain columns |
| `skiprows` | Rows to skip |
| `nrows` | Number of rows to read at a time (combine with `skiprows` to read a file bit by bit) |
| `parse_dates` | Automatically parse columns containing dates into datetime objects |
| `chunksize` | For reading the file in chunks |
| `compression` | For reading in compressed files without extracting beforehand |
| `encoding` | Specifies the file encoding |

## Writing a `DataFrame` Object to a CSV File
Note that the index of `df` is just row numbers, so we don't want to keep it. Therefore, we pass `index=False` to the `to_csv()` method:

In [ ]:
!pwd

In [ ]:
df.to_csv('output2.csv', index=False)

## Writing a `DataFrame` Object to a Database
Note the `if_exists` parameter. By default, it will give you an error if you try to write a table that already exists. Here, we don't care if it is overwritten. Lastly, if we are interested in appending new rows, we set that to `'append'`.

In [ ]:
import sqlite3

with sqlite3.connect('data/quakes.db') as connection:
    pd.read_csv('data/tsunamis.csv').to_sql(
        'tsunamis', connection, index=False, if_exists='replace'
    )

## Creating a `DataFrame` Object by Querying a Database
Using a SQLite database. Otherwise you need to install [SQLAlchemy](https://www.sqlalchemy.org/).

In [ ]:
import sqlite3

with sqlite3.connect('data/quakes.db') as connection:
    tsunamis = pd.read_sql('SELECT * FROM tsunamis', connection)

tsunamis.head()

# **2-3. making dataframe from api**

In [ ]:
import datetime as dt
import pandas as pd
import requests

yesterday = dt.date.today() - dt.timedelta(days=1)
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
payload = {
    'format': 'geojson',
    'starttime': yesterday - dt.timedelta(days=30),
    'endtime': yesterday
}
response = requests.get(api, params=payload)

# let's make sure the request was OK
response.status_code

Response of 200 means OK, so we can pull the data out of the result. Since we asked the API for a JSON payload, we can extract it from the response with the `json()` method.

### Isolate the Data from the JSON Response
We need to check the structures of the response data to know where our data is.

In [ ]:
earthquake_json = response.json()
earthquake_json.keys()

The USGS API provides information about our request in the `metadata` key. Note that your result will be different, regardless of the date range you chose, because the API includes a timestamp for when the data was pulled:

In [ ]:
earthquake_json['metadata']

Each element in the JSON array `features` is a row of data for our dataframe.

In [ ]:
type(earthquake_json['features'])

Your data will be different depending on the date you run this.

In [ ]:
earthquake_json['features']

### Convert to DataFrame
We need to grab the `properties` section out of every entry in the `features` JSON array to create our dataframe.

In [ ]:
earthquake_properties_data = [
    quake['properties'] for quake in earthquake_json['features']
]
earthquake_properties_data

In [ ]:
earthquake_properties_data = [
    quake['properties'] for quake in earthquake_json['features']
]
df = pd.DataFrame(earthquake_properties_data)
df.head()

### (Optional) Write Data to CSV

In [ ]:
df.to_csv('earthquakes.csv', index=False)

# **2-4. inspecting dataframe**

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv('data/earthquakes.csv')

## Examining dataframes
### Is it empty?

In [ ]:
df.empty

### What are the dimensions?

In [ ]:
df.shape

### What columns do we have?
We know there are 26 columns, but what are they? Let's use the `columns` attribute to see:

In [ ]:
df.columns

### What does the data look like?
View rows from the top with `head()`:

In [ ]:
df.head(2)

View rows from the bottom with `tail()`. Let's view 2 rows:

In [ ]:
df.tail(2)

*Tip: we can modify the display options in order to see more columns:*

```python
# check the max columns setting
>>> pd.get_option('display.max_columns')
20

# set the max columns to show when printing the dataframe to 26
>>> pd.set_option('display.max_columns', 26)
# OR
>>> pd.options.display.max_columns = 26

# reset the option
>>> pd.reset_option('display.max_columns')

# get information on all display settings
>>> pd.describe_option('display')
```

*More information can be found in the documentation [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/options.html).*

### What data types do we have?

In [ ]:
df.dtypes

### Getting extra info and finding nulls

In [ ]:
df.info()

## Describing and Summarizing
### Get summary statistics

In [ ]:
df.describe()

Specifying the 5<sup>th</sup> and 95<sup>th</sup> percentile:

In [ ]:
df.describe(percentiles=[0.05, 0.95])

Describe specific data types:

In [ ]:
df.describe(include=np.object)

Or describe all of them:

In [ ]:
df.describe(include='all')

This works on columns also:

In [ ]:
df.felt.describe()

There are methods for specific statistics as well. Here is a sampling of them:

| Method | Description | Data types |
| --- | --- | --- |
| `count()` | The number of non-null observations | Any |
| `nunique()` | The number of unique values | Any |
| `sum()` | The total of the values | Numerical or Boolean |
| `mean()` | The average of the values | Numerical or Boolean |
| `median()` | The median of the values | Numerical |
| `min()` | The minimum of the values | Numerical |
| `idxmin()` | The index where the minimum values occurs | Numerical |
| `max()` | The maximum of the values | Numerical |
| `idxmax()` | The index where the maximum value occurs | Numerical |
| `abs()` | The absolute values of the data | Numerical |
| `std()` | The standard deviation | Numerical |
| `var()` | The variance |  Numerical |
| `cov()` | The covariance between two `Series`, or a covariance matrix for all column combinations in a `DataFrame` | Numerical |
| `corr()` | The correlation between two `Series`, or a correlation matrix for all column combinations in a `DataFrame` | Numerical |
| `quantile()` | Calculates a specific quantile | Numerical |
| `cumsum()` | The cumulative sum | Numerical or Boolean |
| `cummin()` | The cumulative minimum | Numerical |
| `cummax()` | The cumulative maximum | Numerical |

For example, finding the unique values in the `alert` column:

In [ ]:
df.alert.unique()

We can then use `value_counts()` to see how many of each unique value we have:

In [ ]:
df.mag.argmax()

Note that `Index` objects also have several methods to help describe and summarize our data:

| Method | Description |
| --- | --- |
| `argmax()`/`argmin()` | Find the location of the maximum/minimum value in the index |
| `equals()` | Compare the index to another `Index` object for equality |
| `isin()` | Check if the index values are in a list of values and return an array of Booleans |
| `max()`/`min()` | Find the maximum/minimum value in the index |
| `nunique()` | Get the number of unique values in the index |
| `to_series()` | Create a `Series` object from the index |
| `unique()` | Find the unique values of the index |
| `value_counts()`| Create a frequency table for the unique values in the index |

# **2-5. subsetting data**

In [ ]:
import pandas as pd

df = pd.read_csv('data/earthquakes.csv')

## Selecting columns
Grab an entire column using attribute notation:

In [ ]:
df.mag

Grab an entire column using dictionary syntax:

In [ ]:
df['mag']

Selecting multiple columns:

In [ ]:
df[['mag', 'title']]

Selecting columns using list comprehensions and string operations:

In [ ]:
df[
    ['title', 'time'
     ,'mag','magType']
]

In [ ]:
df[
    ['title', 'time']
    + [col for col in df.columns if col.startswith('mag')]
]

Breaking down this example:
1. the list comprehension

In [ ]:
[col for col in df.columns if col.startswith('mag')]

2. assembling the list

In [ ]:
['title', 'time'] \
+ [col for col in df.columns if col.startswith('mag')]

3. using this list as the list of columns

In [ ]:
df[
    ['title', 'time']
    + [col for col in df.columns if col.startswith('mag')]
]

## Slicing
### Selecting rows
Using row numbers (inclusive of first index, exclusive of last):

In [ ]:
df[100:103]

### Selecting rows and columns with chaining

In [ ]:
df[['title', 'time']][100:103]

Order doesn't matter here:

In [ ]:
df[100:103][['title', 'time']].equals(
    df[['title', 'time']][100:103]
)

So we know how to select rows and columns, but can we update values? Well, if we try using what we have learned so far, we will see the following warning:

In [ ]:
df[110:113]['title']

In [ ]:
df[110:113]['title'] = df[110:113]['title'].str.lower()

Note that it worked here, but `pandas` says we were setting a value on a copy of a slice and that we should use `loc` instead (topic of the following section):

In [ ]:
df[110:113]['title']

## Indexing

Now if we do this with `loc` as the warning suggests, everything goes smoothly. Note we have to lower the end index by one since `loc` is inclusive of endpoints:

In [ ]:
df.loc[110:112, 'title'] = df.loc[110:112, 'title'].str.lower()
df.loc[110:112, 'title']

### Indexing with `loc`
Selection of the format `loc[row_indexer, column_indexer]` where `:` can be used to select all:

In [ ]:
df.loc[:,'title']

We can use `loc` to select specific rows and columns without chaining. If we use row numbers with `loc`, they are now **inclusive** of the end index:

In [ ]:
df.loc[10:15, ['title', 'mag']]

#### Indexing with `iloc`
Exclusive of the endpoint just as Python slicing:

In [ ]:
df.iloc[10:15, [19, 8]]

We can use slicing syntax with `iloc` for both rows and columns:

In [ ]:
df.iloc[10:15, 6:10]

When using `loc`, we can slice on column names. This will be inclusive of the endpoint because you can't be expected to know what the next column name will be. As such, we have multiple ways to achieve the same end goal:

In [ ]:
df.iloc[10:15, 6:10].equals(
    df.loc[10:14, 'gap':'magType']
)

### Looking up scalar values
We used `loc` and `iloc` to grab subsets of the dataframe. However, if we are just interested in the specific value at a given `[row, column]`, then we can use `iat` and `at`. We use `at` with labels:

In [ ]:
df.at[10, 'mag']

...and `iat` with integer indices:

In [ ]:
df.iat[10, 8]

## Filtering
We can filter our dataframes using a **Boolean mask**, which can be made as follows:

In [ ]:
df.mag > 2

To use a mask for selection, we simply place it inside the brackets:

In [ ]:
df[df.mag >= 7.0]

We can use masks with `loc`:

In [ ]:
df.loc[
    df.mag >= 7.0,
    ['alert', 'mag', 'magType', 'title', 'tsunami', 'type']
]

Masks can be created using multiple criteria when combined with bitwise operators `&` for AND and `|` for OR. We must also surround each criterion with parentheses. We can't use `and`/`or` here because we need to evaluate row by row:

In [ ]:
df.loc[
    (df.tsunami == 1) & (df.alert == 'red'),
    ['alert', 'mag', 'magType', 'title', 'tsunami', 'type']
]

An example with an OR condition, which is less restrictive:

In [ ]:
df.loc[
    (df.tsunami == 1) | (df.alert == 'red'),
    ['alert', 'mag', 'magType', 'title', 'tsunami', 'type']
]

Masks can be created from any criteria that results in a Boolean. For example, we can select all earthquakes with the string `Alaska` in the `place` column with a non-null value for the `alert` column. To get non-nulls, we can use the `isnull()` method with the bitwise negation operator (`~`) or the `notnull()` method:

In [ ]:
df.loc[
    (df.place.str.contains('Alaska')) & (df.alert.notnull()),
    ['alert', 'mag', 'magType', 'title', 'tsunami', 'type', 'place']
]

We can even use regular expressions here:

In [ ]:
df.loc[
    (df.place.str.contains('CA')) & (df.mag > 3.8),
    ['alert', 'mag', 'magType', 'title', 'tsunami', 'type','place']
]

We can use the `between()` method to turn 2 individual checks (is less than or equal to some maximum value and is greater than or equal to some minimum value) into a single one. Note this is inclusive of the endpoint by default:

In [ ]:
df.loc[
    df.mag.between(6.5, 7.5),
    ['alert', 'mag', 'magType', 'title', 'tsunami', 'type']
]

We can use the `isin()` method to check for membership in a list of values:

In [ ]:
df.loc[
    df.magType.isin(['mw', 'mwb']),
    ['alert', 'mag', 'magType', 'title', 'tsunami', 'type']
]

We can grab the index of the minimum and maximum values of a given column and use those to select the entire row where they occur:

In [ ]:
[df.mag.idxmin(), df.mag.idxmax()]

In [ ]:
df.loc[
    [df.mag.idxmin(), df.mag.idxmax()],
    ['alert', 'mag', 'magType', 'title', 'tsunami', 'type']
]

Note that there is a `filter()` method, but it doesn't filter the data in the same sense as we discussed in this section. Here are a few things you can do with this method.

- grab columns of a dataframe by passing a list to `items`:

In [ ]:
df.filter(items=['mag', 'magType']).head()

- grab all the columns that contain a string with the `like` parameter:

In [ ]:
df.filter(like='mag').head()

- use regular expressions; here, we select any columns that start with `t`:

In [ ]:
df.filter(regex='^t').head()

- use `filter()` along the rows, by passing in `axis=0`. Here, we will use the `place` column as the index (we will cover `set_index()` in chapter 3):

In [ ]:
df.set_index('place').filter(like='Japan',axis=0).filter(items=['mag', 'magType', 'title']).head()

This also works on `Series` objects and will run on the index:

In [ ]:
df.set_index('place').title.filter(like='Japan').head()

## **2-6.adding and removing data**

In [ ]:
import pandas as pd

df = pd.read_csv(
    'data/earthquakes.csv', 
    usecols=['time', 'title', 'place', 'magType', 'mag', 'alert', 'tsunami']
)

## Creating new data
### Adding new columns
New columns get added to the right of the original columns and can be a single value, which will be **broadcast** along the rows of the dataframe:

In [ ]:
df['source'] = 'USGS API'
df.head()

...or a Boolean mask:

In [ ]:
df['mag_negative'] = df.mag < 0
df.head()

#### Adding the `parsed_place` column
We have an entity recognition problem on our hands with the `place` column. There are several entities that have multiple names in the data (e.g., CA and California, NV and Nevada).

In [ ]:
df.place

In [ ]:
df.place.str.extract(', (.*$)')[0]

In [ ]:
df.place.str.extract(', (.*$)')[0].sort_values().unique()

Replace parts of the `place` names to fit our needs:

In [ ]:
df['parsed_place'] = df.place.str.replace(
    r'.* of ', '', regex=True # remove anything saying <something> of <something>
).str.replace(
    'the ', '' # remove "the "
).str.replace(
    r'CA$', 'California', regex=True # fix California
).str.replace(
    r'NV$', 'Nevada', regex=True # fix Nevada
).str.replace(
    r'MX$', 'Mexico', regex=True # fix Mexico
).str.replace(
    r' region$', '', regex=True # chop off endings with " region"
).str.replace(
    'northern ', '' # remove "northern "
).str.replace(
    'Fiji Islands', 'Fiji' # line up the Fiji places
).str.replace(
    r'^.*, ', '', regex=True # remove anything else extraneous from the beginning
).str.strip() # remove any extra spaces

Now we can use a single name to get all earthquakes for that place (although this still isn't perfect):

In [ ]:
df.parsed_place.sort_values().unique()

#### Using the `assign()` method to create columns
To create many columns at once or update existing columns, we can use `assign()`:

In [ ]:
df.assign(
    in_ca=df.parsed_place.str.endswith('California'),
    in_alaska=df.parsed_place.str.endswith('Alaska')
).sample(5, random_state=0)

With the use of `lambda` functions, the `assign()` method becomes even more powerful. **Lambda functions** are anonymous functions usually defined in one line and for single use. The `assign()` method passes the entire dataframe into the `lambda` function as `x`; from there, we can select the columns `in_ca` and `in_alaska`, which are being created in that same call to `assign()`. Here, we use a `lambda` function to create a new column, `neither`, which tells if the earthquake was neither in Alaska nor California:

In [ ]:
df.assign(
    in_ca=df.parsed_place == 'California',
    in_alaska=df.parsed_place == 'Alaska',
    neither=lambda x: ~x.in_ca & ~x.in_alaska
).sample(5, random_state=0)

#### Concatenation
Say we were working with two separate dataframes, one with earthquakes accompanied by tsunamis and the other with earthquakes without tsunamis. If we wanted to look at earthquakes as a whole, we would want to concatenate the dataframes into a single one:

In [ ]:
tsunami = df[df.tsunami == 1]
no_tsunami = df[df.tsunami == 0]

tsunami.shape, no_tsunami.shape

Concatenating along the row axis (`axis=0`) is equivalent to appending to the bottom. By concatenating our earthquakes with tsunamis and those without tsunamis, we get the full earthquake data set back:

In [ ]:
pd.concat([tsunami, no_tsunami])

Note that the previous result is equivalent to running the `append()` method of the dataframe:

In [ ]:
tsunami.append(no_tsunami).shape

We have been working with a subset of the columns from the CSV file, but suppose that now we want to get some of the columns we ignored when we read in the data. Since we have added new columns in this notebook, we won't want to read in the file and perform those operations again. Instead, we will concatenate along the columns (`axis=1`) to add back what we are missing:

In [ ]:
additional_columns

In [ ]:
additional_columns = pd.read_csv(
    'data/earthquakes.csv', usecols=['tz', 'felt', 'ids']
)
pd.concat([df.head(2), additional_columns.head(2)], axis=1)

Notice what happens if the index doesn't align though:

In [ ]:
additional_columns

In [ ]:
additional_columns = pd.read_csv(
    'data/earthquakes.csv', usecols=['tz', 'felt', 'ids', 'time'], index_col='time'
)
pd.concat([df.head(2), additional_columns.head(2)], axis=1)

If the index doesn't align, we can align it before attempting the concatentation, which we will discuss in chapter 3.

Say we want to join the `tsunami` and `no_tsunami` dataframes, but the `no_tsunami` dataframe has an additional column. The `join` parameter specifies how to handle any overlap in column names (when appending to the bottom) or in row names (when concatenating to the left/right). By default, this is `outer`, so we keep everything; however, if we use `inner`, we will only keep what is in common:

In [ ]:
no_tsunami.columns

In [ ]:
tsunami.columns

In [ ]:
pd.concat(
    [tsunami.head(2), no_tsunami.head(2).assign(type='earthquake')], join='outer'
)

In addition, we use `ignore_index`, since the index doesn't mean anything for us here. This gives us sequential values instead of what we had in the previous result:

In [ ]:
pd.concat(
    [tsunami.head(2), no_tsunami.head(2).assign(type='earthquake')], join='inner', ignore_index=True
)

## Deleting Unwanted Data
Columns can be deleted using dictionary syntax with `del`:

In [ ]:
df.columns

In [ ]:
del df['source']
df.columns

If we don't know whether the column exists, we should use a `try`/`except` block:

In [ ]:
try:
    del df['source']
except KeyError:
    # handle the error here
    print('not there anymore')

We can also use `pop()`. This will allow us to use the series we remove later. Note there will be an error if the key doesn't exist, so we can also use a `try`/`except` here:

In [ ]:
mag_negative = df.pop('mag_negative')
df.columns

In [ ]:
try:
    del df['mag_negative']
except KeyError:
    # handle the error here
    print('not there anymore')

Notice we have a mask in `mag_negative` now:

In [ ]:
mag_negative.value_counts()

Now, we can use `mag_negative` to filter our data:

In [ ]:
df[mag_negative].head()

### Using the `drop()` method
We can drop rows by passing a list of indices to the `drop()` method. Notice in the following example that when asking for the first 2 rows with `head()` we get the 3rd and 4th rows because we dropped the original first 2 with `drop([0, 1])`:

In [ ]:
df.drop([0, 1]).head(2)

The `drop()` method drops along the row axis by default. If we pass in a list of columns with the `columns` argument, we can delete columns:

In [ ]:
cols_to_drop = [
    col for col in df.columns
    if col not in ['alert', 'mag', 'title', 'time', 'tsunami']
]
cols_to_drop

In [ ]:
cols_to_drop = [
    col for col in df.columns
    if col not in ['alert', 'mag', 'title', 'time', 'tsunami']
]
df.drop(columns=cols_to_drop).head()

We also have the option of using `axis=1`:

In [ ]:
df.drop(columns=cols_to_drop).equals(
    df.drop(cols_to_drop, axis=1)
)

By default, `drop()`, along with the majority of `DataFrame` methods, will return a new `DataFrame` object. If we just want to change the one we are working with, we can pass `inplace=True`. This should be used with care:

In [ ]:
df.drop(columns=cols_to_drop, inplace=True)
df.head()

# **3-1. wide and long**

In [ ]:
cd /content/drive/MyDrive/Hands-On-Data-Analysis-with-Pandas-2nd-edition/ch_03/

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

wide_df = pd.read_csv('data/wide_data.csv', parse_dates=['date'])
long_df = pd.read_csv(
    'data/long_data.csv', 
    usecols=['date', 'datatype', 'value'], 
    parse_dates=['date']
)[['date', 'datatype', 'value']] # sort columns

## Wide format
Our variables each have their own column:

In [ ]:
wide_df.head(6)

Describing all the columns is easy:

In [ ]:
wide_df.describe(include='all', datetime_is_numeric=True)

It's easy to graph with `pandas` (covered in chapter 5):

In [ ]:
wide_df.plot(
    x='date', y=['TMAX', 'TMIN', 'TOBS'], figsize=(15, 5), 
    title='Temperature in NYC in October 2018'
).set_ylabel('Temperature in Celsius')
plt.show()

## Long format
Our variable names are now in the `datatype` column and their values are in the `value` column. We now have 3 rows for each date, since we have 3 different `datatypes`:

In [ ]:
long_df.head(6)

Since we have many rows for the same date, using `describe()` is not that helpful:

In [ ]:
long_df.describe(include='all', datetime_is_numeric=True)

In [ ]:
grouped_df = long_df.groupby('datatype')

In [ ]:
grouped_df.count()

In [ ]:
grouped_df.mean()

In [ ]:
grouped_df.describe()

In [ ]:
grouped_df.describe().T

Plotting long format data in `pandas` can be rather tricky. Instead we use `seaborn` (covered in [`ch_06/1-introduction_to_seaborn.ipynb`](../ch_06/1-introduction_to_seaborn.ipynb)):

In [ ]:
import seaborn as sns

sns.set(rc={'figure.figsize': (15, 5)}, style='white')

ax = sns.lineplot(
    data=long_df, x='date', y='value', hue='datatype'
)
ax.set_ylabel('Temperature in Celsius')
ax.set_title('Temperature in NYC in October 2018')
plt.show()

With long data and `seaborn`, we can easily facet our plots:

In [ ]:
sns.set(
    rc={'figure.figsize': (20, 10)}, style='white', font_scale=2
)

g = sns.FacetGrid(long_df, col='datatype', height=10)
g = g.map(plt.plot, 'date', 'value')
g.set_titles(size=25)
g.set_xticklabels(rotation=45)
plt.show()

# **3-2. using the weather api**

In [ ]:
import requests

def make_request(endpoint, payload=None):
    """
    Make a request to a specific endpoint on the weather API
    passing headers and optional payload.
    
    Parameters:
        - endpoint: The endpoint of the API you want to 
                    make a GET request to.
        - payload: A dictionary of data to pass along 
                   with the request.
    
    Returns:
        A response object.
    """
    return requests.get(
        f'https://www.ncdc.noaa.gov/cdo-web/api/v2/{endpoint}',
        headers={
            'token': 'PASTE_TOKEN_HERE'
        },
        params=payload
    )

**Note: the API limits us to 5 requests per second and 10,000 requests per day.**

## See which datasets are available
We can make requests to the `datasets` endpoint to see which datasets are available. We also pass in a dictionary for the payload to get datasets that have data after the start date of October 1, 2018.

In [ ]:
response = make_request('datasets', {'startdate': '2018-10-01'})

Status code of `200` means everything is OK. More codes can be found [here](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).

In [ ]:
response.status_code

Alternatively, we can check the `ok` attribute:

In [ ]:
response.ok

### Get the keys of the result
The result is a JSON payload, which we can access with the `json()` method of our response object. JSON objects can be treated like dictionaries, so we can access the keys just like we would a dictionary:

In [ ]:
payload = response.json()
payload.keys()

The metadata of the response will tell us information about the request and data we got back:

In [ ]:
payload['metadata']

### Figure out what data is in the result
The `results` key contains the data we requested. This is a list of what would be rows in our dataframe. Each entry in the list is a dictionary, so we can look at the keys to get the fields:

In [ ]:
payload['results'][0].keys()

### Parse the result
We don't want all those fields, so we will use a list comphrension to take only the `id` and `name` fields out:

In [ ]:
[(data['id'], data['name']) for data in payload['results']]

## Figure out which data category we want
The `GHCND` data containing daily summaries is what we want. Now we need to make another request to figure out which data categories we want to collect. This is the `datacategories` endpoint. We have to pass the `datasetid` for `GHCND` as the payload so the API knows which dataset we are asking about:

In [ ]:
# get data category id
response = make_request(
    'datacategories', payload={'datasetid': 'GHCND'}
)
response.status_code

Since we know the API gives us a `metadata` and a `results` key in each response, we can see what is in the `results` portion of the JSON payload:

In [ ]:
response.json()['results']

## Grab the data type ID for the temperature category
We will be working with temperatures, so we want the `TEMP` data category. Now, we need to find the `datatypes` to collect. For this, we use the `datatypes` endpoint and provide the `datacategoryid` which was `TEMP`. We also specify a limit for the number of `datatypes` to return with the payload. If there are more than this we can make another request later, but for now, we just want to pick a few out:

In [ ]:
# get data type id
response = make_request(
    'datatypes',
    payload={
        'datacategoryid': 'TEMP', 
        'limit': 100
    }
)
response.status_code

We can grab the `id` and `name` fields for each of the entries in the `results` portion of the data. The fields we are interested in are at the bottom:

In [ ]:
[(datatype['id'], datatype['name']) for datatype in response.json()['results']][-5:] # look at the last 5

## Determine which location category we want
Now that we know which `datatypes` we will be collecting, we need to find the location to use. First, we need to figure out the location category. This is obtained from the `locationcategories` endpoint by passing the `datasetid`:

In [ ]:
# get location category id 
response = make_request(
    'locationcategories', 
    payload={'datasetid': 'GHCND'}
)
response.status_code

We can use `pprint` to print dictionaries in an easier-to-read format. After doing so, we can see there are 12 different location categories, but we are only interested in `CITY`:

In [ ]:
import pprint
pprint.pprint(response.json())

## Get NYC Location ID
In order to find the location ID for New York, we need to search through all the cities available. Since we can ask the API to return the cities sorted, we can use binary search to find New York quickly without having to make many requests or request lots of data at once. The following function makes the first request to see how big the list is and looks at the first value. From there it decides if it needs to move towards the beginning or end of the list by comparing the item we are looking for to others alphabetically. Each time it makes a request it can rule out half of the remaining data to search.

In [ ]:
def get_item(name, what, endpoint, start=1, end=None):
    """
    Grab the JSON payload for a given field by name using binary search.

    Parameters:
        - name: The item to look for.
        - what: Dictionary specifying what the item in `name` is.
        - endpoint: Where to look for the item.
        - start: The position to start at. We don't need to touch this, but the
                 function will manipulate this with recursion.
        - end: The last position of the items. Used to find the midpoint, but
               like `start` this is not something we need to worry about.

    Returns:
        Dictionary of the information for the item if found otherwise 
        an empty dictionary.
    """
    # find the midpoint which we use to cut the data in half each time
    mid = (start + (end or 1)) // 2
    
    # lowercase the name so this is not case-sensitive
    name = name.lower()
    
    # define the payload we will send with each request
    payload = {
        'datasetid': 'GHCND',
        'sortfield': 'name',
        'offset': mid, # we will change the offset each time
        'limit': 1 # we only want one value back
    }
    
    # make our request adding any additional filter parameters from `what`
    response = make_request(endpoint, {**payload, **what})
    
    if response.ok:
        payload = response.json()

        # if response is ok, grab the end index from the response metadata the first time through
        end = end or payload['metadata']['resultset']['count']
        
        # grab the lowercase version of the current name
        current_name = payload['results'][0]['name'].lower()
        
        # if what we are searching for is in the current name, we have found our item
        if name in current_name:
            return payload['results'][0] # return the found item
        else:
            if start >= end: 
                # if our start index is greater than or equal to our end, we couldn't find it
                return {}
            elif name < current_name:
                # our name comes before the current name in the alphabet, so we search further to the left
                return get_item(name, what, endpoint, start, mid - 1)
            elif name > current_name:
                # our name comes after the current name in the alphabet, so we search further to the right
                return get_item(name, what, endpoint, mid + 1, end)    
    else:
        # response wasn't ok, use code to determine why
        print(f'Response not OK, status: {response.status_code}')

When we use binary search to find New York, we find it in just 8 requests despite it being close to the middle of 1,983 entries:

In [ ]:
# get NYC id 
nyc = get_item('New York', {'locationcategoryid': 'CITY'}, 'locations')
nyc

## Get the station ID for Central Park
The most granular data is found at the station level:

In [ ]:
central_park = get_item('NY City Central Park', {'locationid': nyc['id']}, 'stations')
central_park

## Request the temperature data
Finally, we have everything we need to make our request for the New York temperature data. For this, we use the `data` endpoint and provide all the parameters we picked up throughout our exploration of the API:

In [ ]:
# get NYC daily summaries data 
response = make_request(
    'data', 
    {
        'datasetid': 'GHCND',
        'stationid': central_park['id'],
        'locationid': nyc['id'],
        'startdate': '2018-10-01',
        'enddate': '2018-10-31',
        'datatypeid': ['TAVG', 'TMAX', 'TMIN'], # average, max, and min temperature
        'units': 'metric',
        'limit': 1000
    }
)
response.status_code

## Create a DataFrame
The Central Park station only has the daily minimum and maximum temperatures.

In [ ]:
import pandas as pd

df = pd.DataFrame(response.json()['results'])
df.head()

We didn't get `TAVG` because the station doesn't measure that:

In [ ]:
df.datatype.unique()

Despite showing up in the data as measuring it... Real-world data is dirty!

In [ ]:
if get_item(
    'NY City Central Park', {'locationid': nyc['id'], 'datatypeid': 'TAVG'}, 'stations'
):
    print('Found!')

## Using a different station
Let's use LaGuardia airport instead. It contains `TAVG` (average daily temperature):

In [ ]:
laguardia = get_item(
    'LaGuardia', {'locationid': nyc['id']}, 'stations'
)
laguardia

We make our request using the LaGuardia airport station this time.

In [ ]:
# get NYC daily summaries data 
response = make_request(
    'data', 
    {
        'datasetid': 'GHCND',
        'stationid': laguardia['id'],
        'locationid': nyc['id'],
        'startdate': '2018-10-01',
        'enddate': '2018-10-31',
        'datatypeid': ['TAVG', 'TMAX', 'TMIN'], # temperature at time of observation, min, and max
        'units': 'metric',
        'limit': 1000
    }
)
response.status_code

The request was successful, so let's make a dataframe:

In [ ]:
df = pd.DataFrame(response.json()['results'])
df.head()

We should check that we got what we wanted: 31 entries for TAVG, TMAX, and TMIN (1 per day):

In [ ]:
df.datatype.value_counts()

Write the data to a CSV file for use in other notebooks.

In [ ]:
df.to_csv('data/nyc_temperatures.csv', index=False)

# **3-3. cleaning data**

In [ ]:
import pandas as pd

df = pd.read_csv('data/nyc_temperatures.csv')
df.head()

## Renaming Columns
We start out with the following columns:

In [ ]:
df.columns

We want to rename the `value` column to indicate it contains the temperature in Celsius and the `attributes` column to say `flags` since each value in the comma-delimited string is a different flag about the data collection. For this task, we use the `rename()` method and pass in a dictionary mapping the column names to their new names. We pass `inplace=True` to change our original dataframe instead of getting a new one back:

In [ ]:
df.rename(
    columns={
        'value': 'temp_C',
        'attributes': 'flags'
    }, inplace=True
)

Those columns have been successfully renamed:

In [ ]:
df.columns

We can also perform string operations on the column names with `rename()`:

In [ ]:
df.rename(str.upper, axis='columns').columns

## Type Conversion
The `date` column is not currently being stored as a `datetime`:

In [ ]:
df.dtypes

Let's perform the conversion with `pd.to_datetime()`:

In [ ]:
df.loc[:,'date'] = pd.to_datetime(df.date)
df.dtypes

Now we get useful information when we use `describe()` on this column:

In [ ]:
df.date.describe(datetime_is_numeric=True)

We can use `tz_localize()` on a `DatetimeIndex` object to convert to a desired timezone:

In [ ]:
pd.date_range(start='2018-10-25', periods=2, freq='D').tz_localize('EST')

This also works with `Series`/`DataFrame` objects that have an index of type `DatetimeIndex`. Let's read in the CSV again for this example and set the `date` column to be the index and stored as a datetime:

In [ ]:
eastern = pd.read_csv(
    'data/nyc_temperatures.csv', index_col='date', parse_dates=True
).tz_localize('EST')
eastern.head()

We can use `tz_convert()` to convert to another timezone from there. If we convert the Eastern datetimes to UTC, they will now be at 5 AM, since `pandas` will use the offsets to convert:

In [ ]:
eastern.tz_convert('UTC').head()

We can change the period of the index as well. We could change the period to be monthly to make it easier to aggregate later. (Aggregation will be discussed in chapter 4.)

In [ ]:
eastern.tz_localize(None).to_period('M').index

We now get a `PeriodIndex` object, which we can change back into a `DatetimeIndex` object with `to_timestamp()`:

In [ ]:
eastern.tz_localize(None).to_period('M').to_timestamp().index

We can use the `assign()` method for working with multiple columns at once (or creating new ones). Since our `date` column has already been converted, we need to read in the data again:

In [ ]:
df = pd.read_csv('data/nyc_temperatures.csv').rename(
    columns={
        'value': 'temp_C',
        'attributes': 'flags'
    }
)

new_df = df.assign(
    date=pd.to_datetime(df.date),
    temp_F=(df.temp_C * 9/5) + 32
)
new_df.dtypes

The `date` column now has datetimes and the `temp_F` column was added:

In [ ]:
new_df.head()

We can also use `astype()` to perform conversions. Let's create columns of the integer portion of the temperatures in Celsius and Fahrenheit. We will use **lambda functions** (first introduced in *Chapter 2, Working with Pandas DataFrames*), so that we can use the values being created in the `temp_F` column to calculate the `temp_F_whole` column. It is very common (and useful) to use lambda functions with `assign()`:

In [ ]:
df = df.assign(
    date=lambda x: pd.to_datetime(x.date),
    temp_C_whole=lambda x: x.temp_C.astype('int'),
    temp_F=lambda x: (x.temp_C * 9/5) + 32,
    temp_F_whole=lambda x: x.temp_F.astype('int')
)

df.head()

Creating categories:

In [ ]:
df_with_categories = df.assign(
    station=df.station.astype('category'),
    datatype=df.datatype.astype('category')
)
df_with_categories.dtypes

In [ ]:
df_with_categories.describe(include='category')

Our categories have no order, but this is something that `pandas` supports:

In [ ]:
pd.Categorical(
    ['med', 'med', 'low', 'high'], 
    categories=['low', 'med', 'high'],
    ordered=True
)

## Reordering, reindexing, and sorting
Say we want to find the days that reached the hottest temperatures in the weather data; we can sort our values by the `temp_C` column with the largest on top to find this: 

In [ ]:
df[df.datatype == 'TMAX'].sort_values(by='temp_C', ascending=False).head(10)

However, this isn't perfect because we have some ties, and they aren't sorted consistently. In the first tie between the 7th and the 10th, the earlier date comes first, but the opposite is true with the tie between the 4th and the 2nd. We can use other columns to break ties and specify how to sort each with `ascending`. Let's break ties with the date column and show earlier dates before later ones:

In [ ]:
df[df.datatype == 'TMAX'].sort_values(by=['temp_C', 'date'], ascending=[False, True]).head(10)

Notice that the index was jumbled in the past 2 results. Here, our index only stores the row number in the original data, but we may not need to keep track of that information. In this case, we can pass in `ignore_index=True` to get a new index after sorting:

In [ ]:
df[df.datatype == 'TMAX'].sort_values(by=['temp_C', 'date'], ascending=[False, True], ignore_index=True).head(10)

When just looking for the n-largest values, rather than wanting to sort all the data, we can use `nlargest()`:

In [ ]:
df[df.datatype == 'TAVG'].nlargest(n=10, columns='temp_C')

We use `nsmallest()` for the n-smallest values.

In [ ]:
df.nsmallest(n=5, columns=['temp_C', 'date'])

The `sample()` method will give us rows (or columns with `axis=1`) at random. We can provide a seed (`random_state`) to make this reproducible. The index after we do this is jumbled:

In [ ]:
df.sample(5, random_state=0)

We can use `sort_index()` to order it again:

In [ ]:
df.sample(5, random_state=0).sort_index().index

The `sort_index()` method can also sort columns alphabetically:

In [ ]:
df.sort_index(axis=1).head()

This can make selection with `loc` easier for many columns:

In [ ]:
df.sort_index(axis=1).head().loc[:,'temp_C':'temp_F_whole']

We must sort the index to compare two dataframes. If the index is different, but the data is the same, they will be marked not-equal:

In [ ]:
df.equals(df.sort_values(by='temp_C'))

Sorting the index solves this issue:

In [ ]:
df.equals(df.sort_values(by='temp_C').sort_index())

Let's set the `date` column as our index:

In [ ]:
df.set_index('date', inplace=True)
df.head()

Now that we have an index of type `DatetimeIndex`, we can do datetime slicing and indexing. As long as we provide a date format that pandas understands, we can grab the data. To select all of 2018, we simply use `df.loc['2018']`, for the fourth quarter of 2018 we can use `df.loc['2018-Q4']`, grabbing October is as simple as using `df.loc['2018-10']`; these can also be combined to build ranges. Let's grab October 11, 2018 through October 12, 2018 (inclusive of both endpoints)&mdash;note that using `loc[]` is optional for ranges:

In [ ]:
df['2018-10-11':'2018-10-12']

We can also use `reset_index()` to get a fresh index and move our current index into a column for safe keeping. This is especially useful if we had data, such as the date, in the index that we don't want to lose:

In [ ]:
df['2018-10-11':'2018-10-12'].reset_index()

Reindexing allows us to conform our axis to contain a given set of labels. Let's turn to the S&P 500 stock data in the `sp500.csv` file to see an example of this. Notice we only have data for trading days (weekdays, excluding holidays):

In [ ]:
sp = pd.read_csv(
    'data/sp500.csv', index_col='date', parse_dates=True
).drop(columns=['adj_close'])

sp.head(10).assign(
    day_of_week=lambda x: x.index.day_name()
)

If we want to look at the value of a portfolio (group of assets) that trade on different days, we need to handle the mismatch in the index. Bitcoin, for example, trades daily. If we sum up all the data we have for each day (aggregations will be covered in chapter 4, so don't fixate on this part), we get the following:

In [ ]:
bitcoin = pd.read_csv(
    'data/bitcoin.csv', index_col='date', parse_dates=True
).drop(columns=['market_cap'])

# every day's closing price = S&P 500 close + Bitcoin close (same for other metrics)
portfolio = pd.concat([sp, bitcoin], sort=False).groupby(level='date').sum()

portfolio.head(10).assign(
    day_of_week=lambda x: x.index.day_name()
)

It may not be immediately obvious what is wrong with the previous data, but with a visualization we can easily see the cyclical pattern of drops on the days the stock market is closed. (Don't worry about the plotting code too much, we will cover it in depth in chapters 5 and 6).

We will need to import `matplotlib` now:

In [ ]:
import matplotlib.pyplot as plt # we use this module for plotting
from matplotlib.ticker import StrMethodFormatter # for formatting the axis

Now we can see why we need to reindex:

In [ ]:
# plot the closing price from Q4 2017 through Q2 2018
ax = portfolio['2017-Q4':'2018-Q2'].plot(
    y='close', figsize=(15, 5), legend=False,
    title='Bitcoin + S&P 500 value without accounting for different indices'
)

# formatting
ax.set_ylabel('price')
ax.yaxis.set_major_formatter(StrMethodFormatter('${x:,.0f}'))
for spine in ['top', 'right']:
    ax.spines[spine].set_visible(False)

# show the plot
plt.show()

We need to align the index of the S&P 500 to match bitcoin in order to fix this. We will use the `reindex()` method, but by default we get `NaN` for the values that we don't have data for:

In [ ]:
sp.reindex(bitcoin.index).head(10).assign(
    day_of_week=lambda x: x.index.day_name()
)

So now we have rows for every day of the year, but all the weekends and holidays have `NaN` values. To address this, we can specify how to handle missing values with the `method` argument. In this case, we want to forward-fill, which will put the weekend and holiday values as the value they had for the Friday (or end of trading week) before:

In [ ]:
sp.reindex(bitcoin.index, method='ffill').head(10)\
    .assign(day_of_week=lambda x: x.index.day_name())

To isolate the changes happening with the forward-filling, we can use the `compare()` method. It shows us the values that differ across identically-labeled dataframes (same names and same columns). Here, we can see that only weekends and holidays (Monday, January 16, 2017 was MLK day) have values forward-filled. Notice that consecutive days have the same values.

In [ ]:
sp.reindex(bitcoin.index)\
    .compare(sp.reindex(bitcoin.index, method='ffill'))\
    .head(10).assign(day_of_week=lambda x: x.index.day_name())

This isn't perfect though. We probably want 0 for the volume traded and to put the closing price for the open, high, low, and close on the days the market is closed:

In [ ]:
import numpy as np

sp_reindexed = sp.reindex(bitcoin.index).assign(
    volume=lambda x: x.volume.fillna(0), # put 0 when market is closed
    close=lambda x: x.close.fillna(method='ffill'), # carry this forward
    # take the closing price if these aren't available
    open=lambda x: np.where(x.open.isnull(), x.close, x.open),
    high=lambda x: np.where(x.high.isnull(), x.close, x.high),
    low=lambda x: np.where(x.low.isnull(), x.close, x.low)
)
sp_reindexed.head(10).assign(
    day_of_week=lambda x: x.index.day_name()
)

If we create a visualization comparing the reindexed data to the first attempt, we see how reindexing helped maintain the asset value when the market was closed:

In [ ]:
# every day's closing price = S&P 500 close adjusted for market closure + Bitcoin close (same for other metrics)
fixed_portfolio = sp_reindexed + bitcoin

# plot the reindexed portfolio's closing price from Q4 2017 through Q2 2018
ax = fixed_portfolio['2017-Q4':'2018-Q2'].plot(
    y='close', label='reindexed portfolio of S&P 500 + Bitcoin', figsize=(15, 5), linewidth=2, 
    title='Reindexed portfolio vs. portfolio with mismatched indices'
)

# add line for original portfolio for comparison
portfolio['2017-Q4':'2018-Q2'].plot(
    y='close', ax=ax, linestyle='--', label='portfolio of S&P 500 + Bitcoin w/o reindexing'
)

# formatting
ax.set_ylabel('price')
ax.yaxis.set_major_formatter(StrMethodFormatter('${x:,.0f}'))
for spine in ['top', 'right']:
    ax.spines[spine].set_visible(False)

# show the plot
plt.show()

# **3-4. reshaping data**

In [ ]:
long_data=pd.read_csv('data/long_data.csv')
long_data

In [ ]:
import pandas as pd

long_df = pd.read_csv(
    'data/long_data.csv', usecols=['date', 'datatype', 'value']
).rename(
    columns={'value': 'temp_C'}
).assign(
    date=lambda x: pd.to_datetime(x.date),
    temp_F=lambda x: (x.temp_C * 9/5) + 32
)
long_df.head()

## Transposing
Transposing swaps the rows and the columns. We use the `T` attribute to do so:

In [ ]:
long_df.set_index('date').T

In [ ]:
long_df.set_index('date').head(6).T

## Pivoting
Going from long to wide format.

### `pivot()`
We can restructure our data by picking a column to go in the index (`index`), a column whose unique values will become column names (`columns`), and the values to place in those columns (`values`). The `pivot()` method can be used when we don't need to perform any aggregation in addition to our restructuring (when our index is unique); if this is not the case, we need the `pivot_table()` method which we will cover in chapter 4. 

In [ ]:
pivoted_df = long_df.pivot(
    index='date', columns='datatype', values='temp_C'
)
pivoted_df.head()

Now that the data is pivoted, we have wide format data that we can grab summary statistics with:

In [ ]:
pivoted_df.describe()

We can also provide multiple values to pivot on, which will result in a hierarchical index:

In [ ]:
pivoted_df = long_df.pivot(
    index='date', columns='datatype', values=['temp_C', 'temp_F']
)
pivoted_df.head()

With the hierarchical index, if we want to select `TMIN` in Fahrenheit, we will first need to select `temp_F` and then `TMIN`:

In [ ]:
pivoted_df['temp_F']['TMIN'].head()

### `unstack()`

We have been working with a single index throughout this chapter; however, we can create an index from any number of columns with `set_index()`. This gives us an index of type `MultiIndex`, where the outermost level corresponds to the first element in the list provided to `set_index()`:

In [ ]:
multi_index_df = long_df.set_index(['date', 'datatype'])
multi_index_df.head().index

Notice there are now 2 index sections of the dataframe:

In [ ]:
multi_index_df.head()

With an index of type `MultiIndex`, we can no longer use `pivot()`. We must now use `unstack()`, which by default moves the innermost index onto the columns:

In [ ]:
unstacked_df = multi_index_df.unstack()
unstacked_df.head()

The `unstack()` method also provides the `fill_value` parameter, which let's us fill-in any `NaN` values that might arise from this restructuring of the data. Consider the case that we have data for the average temperature on October 1, 2018, but no other date:

In [ ]:
extra_data = long_df.append([{
    'datatype': 'TAVG', 
    'date': '2018-10-01', 
    'temp_C': 10, 
    'temp_F': 50
}]).set_index(['date', 'datatype'])

extra_data

In [ ]:
extra_data = long_df.append([{
    'datatype': 'TAVG', 
    'date': '2018-10-01', 
    'temp_C': 10, 
    'temp_F': 50
}]).set_index(['date', 'datatype']).sort_index()

extra_data['2018-10-01':'2018-10-02']

If we use `unstack()` in this case, we will have `NaN` for the `TAVG` columns every day but October 1, 2018:

In [ ]:
extra_data.unstack().head()

To address this, we can pass in an appropriate `fill_value`. However, we are restricted to passing in a value for this, not a strategy (like we saw with `fillna()`), so while `-40` is definitely not be the best value, we can use it to illustrate how this works, since this is the temperature at which Fahrenheit and Celsius are equal:

In [ ]:
extra_data.unstack(fill_value=-40).head()

## Melting
Going from wide to long format.

### Setup

In [ ]:
wide_df = pd.read_csv('data/wide_data.csv')
wide_df.head()

### `melt()`
In order to go from wide format to long format, we use the `melt()` method. We have to specify:
- `id_vars`: which column(s) uniquely identify a row in the wide format (`date`, here)
- `value_vars`: the column(s) that contain(s) the values (`TMAX`, `TMIN`, and `TOBS`, here)

Optionally, we can also provide:
- `value_name`: what to call the column that will contain all the values once melted
- `var_name`: what to call the column that will contain the names of the variables being measured

In [ ]:
melted_df = wide_df.melt(
    id_vars='date',
    value_vars=['TMAX', 'TMIN', 'TOBS'],
    value_name='temp_C',
    var_name='measurement'
)
melted_df.head()

### `stack()`
Another option is `stack()`, which will pivot the columns of the dataframe into the innermost level of the index (resulting in an index of type `MultiIndex`). To illustrate this, let's set our index to be the `date` column:

In [ ]:
wide_df.set_index('date', inplace=True)
wide_df.head()

By running `stack()` now, we will create a second level in our index which will contain the column names of our dataframe (`TMAX`, `TMIN`, `TOBS`). This will leave us with a `Series` object containing the values:

In [ ]:
stacked_series = wide_df.stack()
stacked_series.head()

We can use the `to_frame()` method on our `Series` object to turn it into a `DataFrame` object. Since the series doesn't have a name at the moment, we will pass in the name as an argument:

In [ ]:
stacked_df = stacked_series.to_frame('values')
stacked_df.head()

Once again, we have an index of type `MultiIndex`:

In [ ]:
stacked_df.head().index

Unfortunately, we don't have a name for the `datatype` level:

In [ ]:
stacked_df.index.names

We can use `set_names()` to address this though:

In [ ]:
stacked_df.index.set_names(['date', 'datatype'], inplace=True)
stacked_df.index.names

# **3-5. handling data issues**

# Handling duplicate, missing, or invalid data

## About the data
In this notebook, we will using daily weather data that was taken from the [National Centers for Environmental Information (NCEI) API](https://www.ncdc.noaa.gov/cdo-web/webservices/v2) and altered to introduce many common problems faced when working with data. 

*Note: The NCEI is part of the National Oceanic and Atmospheric Administration (NOAA) and, as you can see from the URL for the API, this resource was created when the NCEI was called the NCDC. Should the URL for this resource change in the future, you can search for "NCEI weather API" to find the updated one.*

## Background on the data

Data meanings:
- `PRCP`: precipitation in millimeters
- `SNOW`: snowfall in millimeters
- `SNWD`: snow depth in millimeters
- `TMAX`: maximum daily temperature in Celsius
- `TMIN`: minimum daily temperature in Celsius
- `TOBS`: temperature at time of observation in Celsius
- `WESF`: water equivalent of snow in millimeters

Some important facts to get our bearings:
- According to the National Weather Service, the coldest temperature ever recorded in Central Park was -15°F (-26.1°C) on February 9, 1934: [source](https://www.weather.gov/media/okx/Climate/CentralPark/extremes.pdf) 
- The temperature of the Sun's photosphere is approximately 5,505°C: [source](https://en.wikipedia.org/wiki/Sun)

## Setup
We need to import `pandas` and read in the dirty data to get started:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/MyDrive/Hands-On-Data-Analysis-with-Pandas-2nd-edition/ch_03

In [ ]:
import pandas as pd

df = pd.read_csv('data/dirty_data.csv')

## Finding problematic data
A good first step is to look at some rows:

In [ ]:
df.head()

Looking at summary statistics can reveal strange or missing values:

In [ ]:
df.describe()

The `info()` method can pinpoint missing values and wrong data types:

In [ ]:
df.info()

We can use the `isna()`/`isnull()` method of the series to find nulls:

In [ ]:
contain_nulls = df[
    df.SNOW.isna() | df.SNWD.isna() | df.TOBS.isna()
    | df.WESF.isna() | df.inclement_weather.isna()
]
contain_nulls.shape[0]

In [ ]:
contain_nulls.head(10)

Note that we can't check if we have `NaN` like this:

In [ ]:
df[df.inclement_weather == 'NaN'].shape[0]

This is because it is actually `np.nan`. However, notice this also doesn't work:

In [ ]:
import numpy as np
df[df.inclement_weather == np.nan].shape[0]

We have to use one of the methods discussed earlier for this to work:

In [ ]:
df[df.inclement_weather.isna()].shape[0]

We can find `-inf`/`inf` by comparing to `-np.inf`/`np.inf`:

In [ ]:
df[df.SNWD.isin([-np.inf, np.inf])]

Rather than do this for each column, we can write a function that will use a [dictionary comprehension](https://www.python.org/dev/peps/pep-0274/) to check all the columns for us:

In [ ]:
def get_inf_count(df):
    """Find the number of inf/-inf values per column in the dataframe"""
    return {
        col: df[df[col].isin([np.inf, -np.inf])].shape[0] for col in df.columns
    }

get_inf_count(df)

Before we can decide how to handle the infinite values of snow depth, we should look at the summary statistics for snowfall, which forms a big part in determining the snow depth:

In [ ]:
pd.DataFrame({
    'np.inf Snow Depth': df[df.SNWD == np.inf].SNOW.describe(),
    '-np.inf Snow Depth': df[df.SNWD == -np.inf].SNOW.describe()
}).T

Let's now look into the `date` and `station` columns. We saw the `?` for station earlier, so we know that was the other unique value. However, we see that some dates are present 8 times in the data and we only have 324 days meaning we are also missing days:

In [ ]:
df.describe(include='object')

We can use the `duplicated()` method to find duplicate rows:

In [ ]:
df[df.duplicated()].shape[0]

The default for `keep` is `'first'` meaning it won't show the first row that the duplicated data was seen in; we can pass in `False` to see it though:

In [ ]:
df[df.duplicated(keep=False)]

We can also specify the columns to use:

In [ ]:
df[df.duplicated(['date', 'station'])]

Let's look at a few duplicates. Just in the few values we see here, we know that the top 4 are actually in the data 6 times because by default we aren't seeing their first occurrence:

In [ ]:
df[df.duplicated()].head()

## Mitigating Issues

### Handling duplicated data
Since we know we have NY weather data and noticed we only had two entries for `station`, we may decide to drop the `station` column because we are only interested in the weather data. However, when dealing with duplicate data, we need to think of the ramifications of removing it. Notice we only have data for the `WESF` column when the station is `?`:

In [ ]:
df[df.WESF.notna()].station.unique()

If we determine it won't impact our analysis, we can use `drop_duplicates()` to remove them:

In [ ]:
# 1. make the date a datetime
df.date = pd.to_datetime(df.date)

# 2. save this information for later
station_qm_wesf = df[df.station == '?'].drop_duplicates('date').set_index('date').WESF

# 3. sort ? to the bottom
df.sort_values('station', ascending=False, inplace=True)

# 4. drop duplicates based on the date column keeping the first occurrence 
# which will be the valid station if it has data
df_deduped = df.drop_duplicates('date')

# 5. remove the station column because we are done with it
df_deduped = df_deduped.drop(columns='station').set_index('date').sort_index()

# 6. take valid station's WESF and fall back on station ? if it is null
df_deduped = df_deduped.assign(
    WESF=lambda x: x.WESF.combine_first(station_qm_wesf)
)

df_deduped

In [ ]:
station_qm_wesf

Here we used the `combine_first()` method to coalesce the values to the first non-null entry; this means that if we had data from both stations, we would first take the value provided by the named station and if (and only if) that station was null would we take the value from the station named `?`. The following table contains some examples of how this would play out:

| station GHCND:USC00280907 | station ? | result of `combine_first()` |
| :---: | :---: | :---: |
| 1 | 17 | 1 |
| 1 | `NaN` | 1 |
| `NaN` | 17 | 17 |
| `NaN` | `NaN` | `NaN` |

Check out the 4th row&mdash;we have `WESF` in the correct spot thanks to the index:

In [ ]:
df_deduped.head()

### Dealing with nulls
We could drop nulls, replace them with some arbitrary value, or impute them using the surrounding data. Each of these options may have ramifications, so we must choose wisely.

We can use `dropna()` to drop rows where any column has a null value. The default options leave us hardly any data:

In [ ]:
df_deduped.dropna().shape

If we pass `how='all'`, we can choose to only drop rows where everything is null, but this removes nothing:

In [ ]:
df_deduped.dropna(how='all').shape

We can use just a subset of columns to determine what to drop with the `subset` argument:

In [ ]:
df_deduped.dropna(
    how='all', subset=['inclement_weather', 'SNOW', 'SNWD']
).shape

This can also be performed along columns, and we can also require a certain number of null values before we drop the data:

In [ ]:
df_deduped.dropna(axis='columns', thresh=df_deduped.shape[0] * .75)

We can choose to fill in the null values instead with `fillna()`:

In [ ]:
df_deduped.loc[:,'WESF'].fillna(0, inplace=True)
df_deduped.head()

At this point we have done everything we can without distorting the data. We know that we are missing dates, but if we reindex, we don't know how to fill in the `NaN` data. With the weather data, we can't assume because it snowed one day that it will snow the next or that the temperature will be the same. For this reason, note that the next few examples are just for illustrative purposes only—just because we can do something doesn't mean we should.

That being said, let's try to address some of remaining issues with the temperature data. We know that when `TMAX` is the temperature of the Sun, it must be because there was no measured value, so let's replace it with `NaN`. We will also do so for `TMIN` which currently uses -40°C for its placeholder when we know that the coldest temperature ever recorded in NYC was -15°F (-26.1°C) on February 9, 1934:

In [ ]:
df_deduped = df_deduped.assign(
    TMAX=lambda x: x.TMAX.replace(5505, np.nan),
    TMIN=lambda x: x.TMIN.replace(-40, np.nan),
)

df_deduped

We will also make an assumption that the temperature won't change drastically day-to-day. Note that this is actually a big assumption, but it will allow us to understand how `fillna()` works when we provide a strategy through the `method` parameter. The `fillna()` method gives us 2 options for the `method` parameter:
- `'ffill'` to forward-fill
- `'bfill'` to back-fill

*Note that `'nearest'` is missing because we are not reindexing.*

Here, we will use `'ffill'` to show how this works:

In [ ]:
df_deduped.assign(
    TMAX=lambda x: x.TMAX.fillna(method='ffill'),
    TMIN=lambda x: x.TMIN.fillna(method='ffill'),
    TOBS=lambda x: x.TOBS.fillna(method='ffill')
).head()

We can use `np.nan_to_num()` to turn `np.nan` into 0 and `-np.inf`/`np.inf` into large negative or positive finite numbers:

In [ ]:
df_deduped.assign(
    SNWD=lambda x: np.nan_to_num(x.SNWD)
).head()

Depending on the data we are working with, we can use the `clip()` method as an alternative to `np.nan_to_num()`. The `clip()` method makes it possible to cap values at a specific minimum and/or maximum threshold. Since `SNWD` can't be negative, let's use `clip()` to enforce a lower bound of zero. To show how the upper bound works, let's use the value of `SNOW`:

In [ ]:
df_deduped.assign(
    SNWD=lambda x: x.SNWD.clip(0, x.SNOW)
).head()

We can couple `fillna()` with other types of calculations. Here we replace missing values of `TMAX` with the median of all `TMAX` values, `TMIN` with the median of all `TMIN` values, and `TOBS` to the average of the `TMAX` and `TMIN` values. Since we place `TOBS` last, we have access to the imputed values for `TMIN` and `TMAX` in the calculation:

In [ ]:
df_deduped.assign(
    TMAX=lambda x: x.TMAX.fillna(x.TMAX.median()),
    TMIN=lambda x: x.TMIN.fillna(x.TMIN.median()),
    # average of TMAX and TMIN
    TOBS=lambda x: x.TOBS.fillna((x.TMAX + x.TMIN) / 2)
).head()

We can also use `apply()` for running the same calculation across columns. For example, let's fill all missing values with their rolling 7-day median of their values, setting the number of periods required for the calculation to 0 to ensure we don't introduce more extra `NaN` values. Rolling calculations will be covered in chapter 4, so this is a preview:

In [ ]:
df_deduped.apply(
    # rolling calculations will be covered in chapter 4, this is a rolling 7-day median
    # we set min_periods (# of periods required for calculation) to 0 so we always get a result 
    lambda x: x.fillna(x.rolling(7, min_periods=0).median())
).head(10)

The last strategy we could try is interpolation with the `interpolate()` method. We specify the `method` parameter with the interpolation strategy to use. There are many options, but we will stick with the default of `'linear'`, which will treat values as evenly spaced and place missing values in the middle of existing ones. We have some missing data, so we will reindex first. Look at January 9th, which we didn't have before—the values for `TMAX`, `TMIN`, and `TOBS` are the average of values the day prior (January 8th) and the day after (January 10th):

In [ ]:
df_deduped\
    .reindex(pd.date_range('2018-01-01', '2018-12-31', freq='D'))\
    .apply(lambda x: x.interpolate())\
    .head(10)

<hr>

<div style="overflow: hidden; margin-bottom: 10px;">
    <div style="float: left;">
         <a href="./4-reshaping_data.ipynb">
            <button>&#8592; Previous Notebook</button>
        </a>
    </div>
    <div style="float: right;">
        <a href="../../solutions/ch_03/solutions.ipynb">
            <button>Solutions</button>
        </a>
        <a href="../ch_04/1-querying_and_merging.ipynb">
            <button>Chapter 4 &#8594;</button>
        </a>
    </div>
</div>
<hr>

# **4-0. weather data collection**

# Collecting weather data from an API

This notebook contains the code that was used to collect the data for this chapter. Note that if you overwrite the data that came with this chapter by saving the data you collect here, your results in the remaining notebooks may not match the book due to changes in the NCEI API's data.

## About the data
In this notebook, we will be collecting daily weather data from the [National Centers for Environmental Information (NCEI) API](https://www.ncdc.noaa.gov/cdo-web/webservices/v2). We will use the Global Historical Climatology Network - Daily (GHCND) dataset; see the documentation [here](https://www1.ncdc.noaa.gov/pub/data/cdo/documentation/GHCND_documentation.pdf).

*Note: The NCEI is part of the National Oceanic and Atmospheric Administration (NOAA) and, as you can see from the URL for the API, this resource was created when the NCEI was called the NCDC. Should the URL for this resource change in the future, you can search for "NCEI weather API" to find the updated one.*

## Using the NCEI API
Request your token [here](https://www.ncdc.noaa.gov/cdo-web/token) and then paste it below.

In [ ]:
import requests

def make_request(endpoint, payload=None):
    """
    Make a request to a specific endpoint on the weather API
    passing headers and optional payload.
    
    Parameters:
        - endpoint: The endpoint of the API you want to 
                    make a GET request to.
        - payload: A dictionary of data to pass along 
                   with the request.
    
    Returns:
        Response object.
    """
    return requests.get(
        f'https://www.ncdc.noaa.gov/cdo-web/api/v2/{endpoint}',
        headers={
            'token': 'PASTE_YOUR_TOKEN_HERE'
        },
        params=payload
    )

## Collect All Data Points for 2018 In NYC (Various Stations)
We can make a loop to query for all the data points one day at a time, providing updates using `IPython.display`. Here we create a list of all the results:

In [ ]:
import datetime

from IPython import display # for updating the cell dynamically

current = datetime.date(2018, 1, 1)
end = datetime.date(2019, 1, 1)

results = []

while current < end:
    # update the cell with status information
    display.clear_output(wait=True)
    display.display(f'Gathering data for {str(current)}')
    
    response = make_request(
        'data', 
        {
            'datasetid': 'GHCND', # Global Historical Climatology Network - Daily (GHCND) dataset
            'locationid': 'CITY:US360019', # NYC
            'startdate': current,
            'enddate': current,
            'units': 'metric',
            'limit': 1000 # max allowed
        }
    )

    if response.ok:
        # we extend the list instead of appending to avoid getting a nested list
        results.extend(response.json()['results'])

    # update the current date to avoid an infinite loop
    current += datetime.timedelta(days=1)

Now, we can create a dataframe with all this data. Notice there are multiple stations with values for each `datatype` on a given day. We don't know what the stations are, but we can look them up and add them to the data:

In [ ]:
import pandas as pd

df = pd.DataFrame(results)
df.head()

Save this data to a file:

In [ ]:
df.to_csv('data/nyc_weather_2018.csv', index=False)

and write it to the database:

In [ ]:
import sqlite3

with sqlite3.connect('data/weather.db') as connection:
    df.to_sql(
        'weather', connection, index=False, if_exists='replace'
    )

For learning about merging dataframes, we will also get the data mapping station IDs to information about the station:

In [ ]:
response = make_request(
    'stations', 
    {
        'datasetid': 'GHCND', # Global Historical Climatology Network - Daily (GHCND) dataset
        'locationid': 'CITY:US360019', # NYC
        'limit': 1000 # max allowed
    }
)

stations = pd.DataFrame(response.json()['results'])[['id', 'name', 'latitude', 'longitude', 'elevation']]
stations.to_csv('data/weather_stations.csv', index=False)

with sqlite3.connect('data/weather.db') as connection:
    stations.to_sql(
        'stations', connection, index=False, if_exists='replace'
    )

<hr>
<div>
    <a href="../ch_03/5-handling_data_issues.ipynb">
        <button>&#8592; Chapter 3</button>
    </a>
    <a href="./1-querying_and_merging.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<hr>

# **4-1. querying and merging**

# Performing Database-style Operations on Dataframes

## About the data
In this notebook, we will using daily weather data that was taken from the [National Centers for Environmental Information (NCEI) API](https://www.ncdc.noaa.gov/cdo-web/webservices/v2). The [`0-weather_data_collection.ipynb`](./0-weather_data_collection.ipynb) notebook contains the process that was followed to collect the data. Consult the dataset's [documentation](https://www1.ncdc.noaa.gov/pub/data/cdo/documentation/GHCND_documentation.pdf) for information on the fields.

*Note: The NCEI is part of the National Oceanic and Atmospheric Administration (NOAA) and, as you can see from the URL for the API, this resource was created when the NCEI was called the NCDC. Should the URL for this resource change in the future, you can search for "NCEI weather API" to find the updated one.*


## Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/MyDrive/Hands-On-Data-Analysis-with-Pandas-2nd-edition/ch_04

In [ ]:
import pandas as pd

weather = pd.read_csv('data/nyc_weather_2018.csv')
weather.head()

## Querying DataFrames
The `query()` method is an easier way of filtering based on some criteria. For example, we can use it to find all entries where snow was recorded from a station with `US1NY` in its station ID:

In [ ]:
snow_data = weather.query('datatype == "SNOW" and value > 0 and station.str.contains("US1NY")',engine='python')
snow_data.head()

In [ ]:
weather[
    (weather.datatype == 'SNOW')
    & (weather.value > 0)
    & (weather.station.str.contains('US1NY'))
]

This is equivalent to querying the `weather.db` SQLite database for 

```sql
SELECT * 
FROM weather 
WHERE datatype == "SNOW" AND value > 0 AND station LIKE "%US1NY%"
```

In [ ]:
import sqlite3

with sqlite3.connect('data/weather.db') as connection:
    snow_data_from_db = pd.read_sql(
        'SELECT * FROM weather WHERE datatype == "SNOW" AND value > 0 and station LIKE "%US1NY%"', 
        connection
    )

snow_data.reset_index().drop(columns='index').equals(snow_data_from_db)

Note this is also equivalent to creating Boolean masks:

In [ ]:
weather[
    (weather.datatype == 'SNOW') 
    & (weather.value > 0)
    & weather.station.str.contains('US1NY')
]

## Merging DataFrames
We have data for many different stations each day; however, we don't know what the stations are&mdash;just their IDs. We can join the data in the `weather_stations.csv` file which contains information from the `stations` endpoint of the NCEI API. Consult the [`0-weather_data_collection.ipynb`](./0-weather_data_collection.ipynb) notebook to see how this was collected. It looks like this:

In [ ]:
station_info = pd.read_csv('data/weather_stations.csv')
station_info.head()

As a reminder, the weather data looks like this:

In [ ]:
weather.head()

We can join our data by matching up the `station_info.id` column with the `weather.station` column. Before doing that though, let's see how many unique values we have:

In [ ]:
station_info.id.describe()

While `station_info` has one row per station, the `weather` dataframe has many entries per station. Notice it also has fewer uniques:

In [ ]:
weather.station.describe()

When working with joins, it is important to keep an eye on the row count. Some join types will lead to data loss. Remember that we can get this with `shape`:

In [ ]:
station_info.shape[0], weather.shape[0]

Since we will be doing this often, it makes more sense to write a function:

In [ ]:
def get_row_count(*dfs):
    return [df.shape[0] for df in dfs]
get_row_count(station_info, weather)

By default, `merge()` performs an inner join. We simply specify the columns to use for the join. The left dataframe is the one we call `merge()` on, and the right one is passed in as an argument:

In [ ]:
inner_join = weather.merge(station_info, left_on='station', right_on='id')
inner_join

We can remove the duplication of information in the `station` and `id` columns by renaming one of them before the merge and then simply using `on`:

In [ ]:
weather.merge(station_info.rename(dict(id='station'), axis=1), on='station').sample(5, random_state=0)

We are losing stations that don't have weather observations associated with them, if we don't want to lose these rows, we perform a right or left join instead of the inner join:

In [ ]:
left_join = station_info.merge(weather, left_on='id', right_on='station', how='left')
right_join = weather.merge(station_info, left_on='station', right_on='id', how='right')

right_join

The left and right join as we performed above are equivalent because the side for which we kept the rows without matches was the same in both cases:

In [ ]:
left_join.sort_index(axis=1).sort_values(['date', 'station'], ignore_index=True).equals(
    right_join.sort_index(axis=1).sort_values(['date', 'station'], ignore_index=True)
)

Note we have additional rows in the left and right joins because we kept all the stations that didn't have weather observations:

In [ ]:
get_row_count(inner_join, left_join, right_join)

If we query the station information for stations that have `US1NY` in their ID and perform an outer join, we can see where the mismatches occur:

In [ ]:
outer_join = weather.merge(
    station_info[station_info.id.str.contains('US1NY')], 
    left_on='station', right_on='id', how='outer', indicator=True
)

pd.concat([
    outer_join.query(f'_merge == "{kind}"').sample(2, random_state=0) 
    for kind in outer_join._merge.unique()
]).sort_index()

These joins are equivalent to their SQL counterparts. Below is the inner join. Note that to use `equals()` you will have to do some manipulation of the dataframes to line them up:

In [ ]:
import sqlite3

with sqlite3.connect('data/weather.db') as connection:
    inner_join_from_db = pd.read_sql(
        'SELECT * FROM weather JOIN stations ON weather.station == stations.id', 
        connection
    )

inner_join_from_db.shape == inner_join.shape

Revisiting the dirty data from chapter 3's [`5-handling_data_issues.ipynb`](../ch_03/5-handling_data_issues.ipynb) notebook.

Data meanings:
- `PRCP`: precipitation in millimeters
- `SNOW`: snowfall in millimeters
- `SNWD`: snow depth in millimeters
- `TMAX`: maximum daily temperature in Celsius
- `TMIN`: minimum daily temperature in Celsius
- `TOBS`: temperature at time of observation in Celsius
- `WESF`: water equivalent of snow in millimeters


Read in the data, dropping duplicates and the uninformative `SNWD` column:

In [ ]:
dirty_data = pd.read_csv(
    'data/dirty_data.csv', index_col='date'
).drop_duplicates().drop(columns='SNWD')
dirty_data.head()

We need to create two dataframes for the join. We will drop some unecessary columns as well for easier viewing:

In [ ]:
valid_station = dirty_data.query('station != "?"').drop(columns=['WESF', 'station'])
station_with_wesf = dirty_data.query('station == "?"').drop(columns=['station', 'TOBS', 'TMIN', 'TMAX'])

Our column for the join is the index in both dataframes, so we must specify `left_index` and `right_index`:

In [ ]:
valid_station.merge(
    station_with_wesf, how='left', left_index=True, right_index=True
).query('WESF > 0').head()

The columns that existed in both dataframes, but didn't form part of the join got suffixes added to their names: `_x` for columns from the left dataframe and `_y` for columns from the right dataframe. We can customize this with the `suffixes` argument:

In [ ]:
valid_station.merge(
    station_with_wesf, how='left', left_index=True, right_index=True, suffixes=('', '_?')
).query('WESF > 0').head()

Since we are joining on the index, an easier way is to use the `join()` method instead of `merge()`. Note that the suffix parameter is now `lsuffix` for the left dataframe's suffix and `rsuffix` for the right one's:

In [ ]:
valid_station.join(station_with_wesf, how='left', rsuffix='_?').query('WESF > 0').head()

Joins can be very resource-intensive, so it's a good idea to figure out what type of join you need using set operations before trying the join itself. The `pandas` set operations are performed on the index, so whichever columns we will be joining on will need to be the index. Let's go back to the `weather` and `station_info` dataframes and set the station ID columns as the index:

In [ ]:
weather.set_index('station', inplace=True)
station_info.set_index('id', inplace=True)

The intersection will tell us the stations that are present in both dataframes. The result will be the index when performing an inner join:

In [ ]:
weather.index.intersection(station_info.index)

The set difference will tell us what we lose from each side. When performing an inner join, we lose nothing from the `weather` dataframe:

In [ ]:
weather.index.difference(station_info.index)

We lose 169 stations from the `station_info` dataframe, however:

In [ ]:
station_info.index.difference(weather.index)

The symmetric difference tells us what we lose from both sides. It is the combination of the set differences in each direction:

In [ ]:
ny_in_name = station_info[station_info.index.str.contains('US1NY')]

ny_in_name.index.difference(weather.index).shape[0]\
+ weather.index.difference(ny_in_name.index).shape[0]\
== weather.index.symmetric_difference(ny_in_name.index).shape[0]

The union will show us everything that will be present after a full outer join. Note that we pass in the unique values of the index to make sure we can see the number of stations we will be left with:

In [ ]:
weather.index.unique().union(station_info.index)

Note that the symmetric difference is actually the union of the set differences:

In [ ]:
ny_in_name = station_info[station_info.index.str.contains('US1NY')]

ny_in_name.index.difference(weather.index).union(weather.index.difference(ny_in_name.index)).equals(
    weather.index.symmetric_difference(ny_in_name.index)
)

<hr>
<div>
    <a href="../ch_03/5-handling_data_issues.ipynb">
        <button>&#8592; Chapter 3</button>
    </a>
    <a href="./0-weather_data_collection.ipynb">
        <button>Weather Data Collection</button>
    </a>
    <a href="./2-dataframe_operations.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<hr>

# **4-2. dataframe operations**

# DataFrame Operations

## About the Data
In this notebook, we will be working with 2 datasets:
- Facebook's stock price throughout 2018 (obtained using the [`stock_analysis` package](https://github.com/stefmolin/stock-analysis)).
- Daily weather data for NYC from the [National Centers for Environmental Information (NCEI) API](https://www.ncdc.noaa.gov/cdo-web/webservices/v2).

*Note: The NCEI is part of the National Oceanic and Atmospheric Administration (NOAA) and, as you can see from the URL for the API, this resource was created when the NCEI was called the NCDC. Should the URL for this resource change in the future, you can search for "NCEI weather API" to find the updated one.*

## Background on the weather data

Data meanings:
- `AWND`: average wind speed
- `PRCP`: precipitation in millimeters
- `SNOW`: snowfall in millimeters
- `SNWD`: snow depth in millimeters
- `TMAX`: maximum daily temperature in Celsius
- `TMIN`: minimum daily temperature in Celsius

## Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/MyDrive/Hands-On-Data-Analysis-with-Pandas-2nd-edition/ch_04

In [ ]:
import numpy as np
import pandas as pd

weather = pd.read_csv('data/nyc_weather_2018.csv', parse_dates=['date'])
weather.head()

In [ ]:
fb = pd.read_csv('data/fb_2018.csv', index_col='date', parse_dates=True)
fb.head()

## Arithmetic and statistics
We already saw that we can use mathematical operators like `+` and `/` with dataframes directly. However, we can also use methods, which allow us to specify the axis to perform the calculation over. By default, this is per column. Let's find the Z-scores for the volume traded and look at the days where this was more than 3 standard deviations from the mean:

In [ ]:
fb.assign(
    abs_z_score_volume=lambda x: \
        x.volume.sub(x.volume.mean()).div(x.volume.std()).abs()
).query('abs_z_score_volume > 3')

We can use `rank()` and `pct_change()` to see which days had the largest change in volume traded from the day before:

In [ ]:
fb.assign(
    volume_pct_change=fb.volume.pct_change(),
    pct_change_rank=lambda x: \
        x.volume_pct_change.abs().rank(ascending=False)
).nsmallest(5, 'pct_change_rank')

January 12th was when the news that Facebook changed its news feed product to focus more on content from a users' friends over the brands they follow. Given that Facebook's advertising is a key component of its business ([nearly 89% in 2017](https://www.investopedia.com/ask/answers/120114/how-does-facebook-fb-make-money.asp)), many shares were sold and the price dropped in panic:

In [ ]:
fb['2018-01-11':'2018-01-12']

Throughout 2018, Facebook's stock price never had a low above $215:

In [ ]:
(fb > 215).any()

Facebook's OHLC (open, high, low, and close) prices all had at least one day they were at $215 or less:

In [ ]:
(fb > 215).all()

## Binning
When working with volume traded, we may be interested in ranges of volume rather than the exact values. No two days have the same volume traded:

In [ ]:
(fb.volume.value_counts() > 1).sum()

We can use `pd.cut()` to create 3 bins of even range in volume traded and name them. Then we can work with low, medium, and high volume traded categories:

In [ ]:
volume_binned = pd.cut(fb.volume, bins=3, labels=['low', 'med', 'high'])
volume_binned.value_counts()

Let's look at the days with high trading volume:

In [ ]:
fb[volume_binned == 'high'].sort_values('volume', ascending=False)

July 25th Facebook announced disappointing user growth and the stock tanked in the after hours:

In [ ]:
fb['2018-07-25':'2018-07-26']

Cambridge Analytica scandal broke on Saturday, March 17th, so we look at the Monday after for the numbers:

In [ ]:
fb['2018-03-16':'2018-03-20']

Since most days have similar volume, but a few are very large, we have very wide bins. Most of the data is in the low bin. 

*Note: visualizations will be covered in chapters 5 and 6.*

In [ ]:
import os
import sys
sys.path.append('/content/drive/MyDrive/Hands-On-Data-Analysis-with-Pandas-2nd-edition/visual-aids/')

In [ ]:
from visual_aids.misc_viz import low_med_high_bins_viz

low_med_high_bins_viz(
    fb, 'volume', ylabel='volume traded',
    title='Daily Volume Traded of Facebook Stock in 2018 (with bins)'
)

If we split using quantiles, the bins will have roughly the same number of observations. For this, we use `qcut()`. We will make 4 quartiles:

In [ ]:
volume_qbinned = pd.qcut(fb.volume, q=4, labels=['q1', 'q2', 'q3', 'q4'])
volume_qbinned.value_counts()

Notice the bins don't cover ranges of the same size anymore:

In [ ]:
from visual_aids.misc_viz import quartile_bins_viz

quartile_bins_viz(
    fb, 'volume', ylabel='volume traded', 
    title='Daily Volume Traded of Facebook Stock in 2018 (with quartile bins)'
)

## Applying Functions
We can use the `apply()` method to run the same operation on all columns (or rows) of the dataframe. First, let's isolate the weather observations from the Central Park station and pivot the data:

In [ ]:
central_park_weather = weather\
    .query('station == "GHCND:USW00094728"')\
    .pivot(index='date', columns='datatype', values='value')

central_park_weather

Let's calculate the Z-scores of the TMIN, TMAX, and PRCP observations in Central Park in October 2018:

In [ ]:
oct_weather_z_scores = central_park_weather\
    .loc['2018-10', ['TMIN', 'TMAX', 'PRCP']]\
    .apply(lambda x: x.sub(x.mean()).div(x.std()))
oct_weather_z_scores

October 27th rained much more than the rest of the days:

In [ ]:
oct_weather_z_scores.query('PRCP > 3').PRCP

Indeed, this day was much higher than the rest:

In [ ]:
central_park_weather.loc['2018-10', 'PRCP'].describe()

When the function we want to apply isn't vectorized, we can:
- use `np.vectorize()` to vectorize it (similar to how `map()` works) and then use it with `apply()`
- use `applymap()` and pass it the non-vectorized function directly

Say we wanted to count the digits of the whole numbers for the Facebook data; `len()` is not vectorized, so we can use `np.vectorize()` or `applymap()`:

In [ ]:
fb.apply(
    lambda x: np.vectorize(lambda y: len(str(np.ceil(y))))(x)
).astype('int64').equals(
    fb.applymap(lambda x: len(str(np.ceil(x))))
)

A simple operation of addition to each element in a series grows linearly in time complexity when using `iteritems()`, but stays near 0 when using vectorized operations. `iteritems()` and related methods should only be used if there is no vectorized solution:

In [ ]:
import time

import numpy as np
import pandas as pd

np.random.seed(0)

vectorized_results = {}
iteritems_results = {}

for size in [10, 100, 1000, 10000, 100000, 500000, 1000000, 5000000, 10000000]:
    # set of numbers to use
    test = pd.Series(np.random.uniform(size=size))
    
    # time the vectorized operation
    start = time.time()
    x = test + 10
    end = time.time()
    vectorized_results[size] = end - start
    
    # time the operation with `iteritems()`
    start = time.time()
    x = []
    for i, v in test.iteritems():
        x.append(v + 10)
    x = pd.Series(x)
    end = time.time()
    iteritems_results[size] = end - start

results = pd.DataFrame(
    [pd.Series(vectorized_results, name='vectorized'), pd.Series(iteritems_results, name='iteritems')]
).T    

# plotting
ax = results.plot(title='Time Complexity', color=['blue', 'red'], legend=False)

# formatting
ax.set(xlabel='item size (rows)', ylabel='time (s)')
ax.text(0.5e7, iteritems_results[0.5e7] * .9, 'iteritems()', rotation=34, color='red', fontsize=12, ha='center', va='bottom')
ax.text(0.5e7, vectorized_results[0.5e7], 'vectorized', color='blue', fontsize=12, ha='center', va='bottom')
for spine in ['top', 'right']:
    ax.spines[spine].set_visible(False)

## Window Calculations
*Consult the [`understanding_window_calculations.ipynb`](./understanding_window_calculations.ipynb) notebook for interactive visualizations using widgets to help understand window calculations.*

The `rolling()` method allows us to perform rolling window calculations. We simply specify the window size (3 days here) and follow it with a call to an aggregation function (sum here):

In [ ]:
central_park_weather.loc['2018-10'].assign(
    rolling_PRCP=lambda x: x.PRCP.rolling('3D').sum()
)[['PRCP', 'rolling_PRCP']].head(7).T

We can also perform the rolling calculations on the entire dataframe at once. This will apply the same aggregation function to each column:

In [ ]:
central_park_weather.loc['2018-10'].rolling('3D').mean().head(7).iloc[:,:6]

We can use different aggregation functions per column if we use `agg()` instead. We pass in a dictionary mapping the column to the aggregation to perform on it. Here, we join the result to the original data to see what is happening:

In [ ]:
central_park_weather['2018-10-01':'2018-10-07'].rolling('3D').agg(
    {'TMAX': 'max', 'TMIN': 'min', 'AWND': 'mean', 'PRCP': 'sum'}
).join( # join with original data for comparison
    central_park_weather[['TMAX', 'TMIN', 'AWND', 'PRCP']], 
    lsuffix='_rolling'
).sort_index(axis=1) # sort columns so rolling calcs are next to originals

Suppose we reindexed the Facebook stock data as we did with the S&P 500 data in chapter 3. If we were to use rolling calculations on this data, we would be including the values when the market was closed:

In [ ]:
fb_reindexed = fb\
    .reindex(pd.date_range('2018-01-01', '2018-12-31', freq='D'))\
    .assign(
        volume=lambda x: x.volume.fillna(0),
        close=lambda x: x.close.fillna(method='ffill'),
        open=lambda x: x.open.combine_first(x.close),
        high=lambda x: x.high.combine_first(x.close),
        low=lambda x: x.low.combine_first(x.close)
    )
fb_reindexed.assign(day=lambda x: x.index.day_name()).head(10)

As of version 1.0, `pandas` supports defining custom windows for rolling calculations, which makes it possible for us to perform rolling calculations on the days the market was open. One way is to make a new class that inherits from `BaseIndexer` and provide the logic for determining the window bounds in the `get_window_bounds()` method (more info [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/computation.html#custom-window-rolling)). For our use case, we can use the `VariableOffsetWindowIndexer` class, which was introduced in version 1.1, to perform rolling calculations over non-fixed time offsets (like business days). Let's perform a three business day rolling calculation on the reindexed Facebook stock data and join it with the reindexed data for comparison:

In [ ]:
from pandas.api.indexers import VariableOffsetWindowIndexer

indexer = VariableOffsetWindowIndexer(
    index=fb_reindexed.index, offset=pd.offsets.BDay(3)
)
fb_reindexed.assign(window_start_day=0).rolling(indexer).agg({
    'window_start_day': lambda x: x.index.min().timestamp(),
    'open': 'mean', 'high': 'max', 'low': 'min',
    'close': 'mean', 'volume': 'sum'
}).join(
    fb_reindexed, lsuffix='_rolling'
).sort_index(axis=1).assign(
    day=lambda x: x.index.day_name(),
    window_start_day=lambda x: pd.to_datetime(x.window_start_day, unit='s')
).head(10)

Rolling calculations (`rolling()`) use a sliding window. Expanding calculations (`expanding()`), however, grow in size. These are equivalent to cumulative aggregations like `cumsum()`; however, we can specify the minimum number of periods required to start calculating (default is 1), and we aren't limited to predefined aggregations. Therefore, while there is no method for the cumulative mean, we can calculate it using `expanding()`. Let's calculate the month-to-date average precipiation:

In [ ]:
central_park_weather.loc['2018-06'].assign(
    TOTAL_PRCP=lambda x: x.PRCP.cumsum(),
    AVG_PRCP=lambda x: x.PRCP.expanding().mean()
).head(10)[['PRCP', 'TOTAL_PRCP', 'AVG_PRCP']].T

We can also use `agg()` to specify aggregations per column. Note that this works with NumPy functions as well. Here, we join the expanding calculations with the original results for comparison:

In [ ]:
central_park_weather['2018-10-01':'2018-10-07'].expanding().agg(
    {'TMAX': np.max, 'TMIN': np.min, 'AWND': np.mean, 'PRCP': np.sum}
).join(
    central_park_weather[['TMAX', 'TMIN', 'AWND', 'PRCP']], 
    lsuffix='_expanding'
).sort_index(axis=1)

Pandas provides the `ewm()` method for exponentially weighted moving calculations. As we saw in chapter 1, we can use the exponentially weighted moving average to smooth the data. Let's compare the rolling mean to the exponentially weighted moving average with the maximum daily temperature. Note that `span` here is the periods to use:

In [ ]:
central_park_weather.assign(
    AVG=lambda x: x.TMAX.rolling('30D').mean(),
    EWMA=lambda x: x.TMAX.ewm(span=30).mean()
).loc['2018-09-29':'2018-10-08', ['TMAX', 'EWMA', 'AVG']].T

*Consult the [`understanding_window_calculations.ipynb`](./understanding_window_calculations.ipynb) notebook for interactive visualizations to help understand window calculations.*

## Pipes
Pipes are a way to streamline our `pandas` code and make it more readable and flexible. Using pipes, we can take a nested call like 

```python
f(g(h(data), 20), x=True)
```

and turn it into something more readable:

```python
data.pipe(h)\
    .pipe(g, 20)\
    .pipe(f, x=True)\
```

We can use pipes to apply any function that accepts our data as the first argument and pass in any additional arguments. This makes it easy to chain steps together regardless of whether they are methods or functions:

We can pass any function that will accept the caller of `pipe()` as the first argument:

In [ ]:
def get_info(df):
    return '%d rows, %d columns and max closing Z-score was %d' % (*df.shape, df.close.max())

get_info(fb.loc['2018-Q1'].apply(lambda x: (x - x.mean())/x.std()))\
    == fb.loc['2018-Q1'].apply(lambda x: (x - x.mean())/x.std()).pipe(get_info)

For example, passing `pd.DataFrame.rolling` to `pipe()` is equivalent to calling `rolling()` directly on the dataframe, except we have more flexiblity to change this:

In [ ]:
fb.pipe(pd.DataFrame.rolling, '20D').mean().equals(fb.rolling('20D').mean())

The pipe takes the function passed in and calls it with the object that called `pipe()` as the first argument. Positional and keyword arguments are passed down:

In [ ]:
pd.DataFrame.rolling(fb, '20D').mean().equals(fb.rolling('20D').mean())

We can use a pipe to make a function that we can use for all of our window calculation needs:

In [ ]:
from window_calc import window_calc
window_calc??

We can use the same interface to calculate various window calculations now. Let's find the expanding median for the Facebook data:

In [ ]:
window_calc(fb, pd.DataFrame.expanding, np.median).head()

Using the exponentially weighted moving average requires we pass in a keyword argument:

In [ ]:
window_calc(fb, pd.DataFrame.ewm, 'mean', span=3).head()

With rolling calculations, we can pass in a positional argument for the window size:

In [ ]:
window_calc(
    central_park_weather.loc['2018-10'], 
    pd.DataFrame.rolling, 
    {'TMAX': 'max', 'TMIN': 'min', 'AWND': 'mean', 'PRCP': 'sum'},
    '3D'
).head()

<hr>
<div style="overflow: hidden; margin-bottom: 10px;">
    <div style="float: left;">
         <a href="./1-querying_and_merging.ipynb">
            <button>&#8592; Previous Notebook</button>
        </a>
        <a href="./understanding_window_calculations.ipynb">
            <button>Understanding Window Calculations</button>
        </a>
    </div>
    <div style="float: right;">
        <a href="./3-aggregations.ipynb">
            <button>Next Notebook &#8594;</button>
        </a>
    </div>
</div>
<hr>

# **4-3. aggregations**

# Aggregating data with pandas and numpy

## About the Data
In this notebook, we will be working with 2 datasets:
- Facebook's stock price throughout 2018 (obtained using the [`stock_analysis` package](https://github.com/stefmolin/stock-analysis)).
- daily weather data for NYC from the [National Centers for Environmental Information (NCEI) API](https://www.ncdc.noaa.gov/cdo-web/webservices/v2).

*Note: The NCEI is part of the National Oceanic and Atmospheric Administration (NOAA) and, as you can see from the URL for the API, this resource was created when the NCEI was called the NCDC. Should the URL for this resource change in the future, you can search for "NCEI weather API" to find the updated one.*

## Background on the weather data

Data meanings:
- `AWND`: average wind speed
- `PRCP`: precipitation in millimeters
- `SNOW`: snowfall in millimeters
- `SNWD`: snow depth in millimeters
- `TMAX`: maximum daily temperature in Celsius
- `TMIN`: minimum daily temperature in Celsius

## Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/MyDrive/Hands-On-Data-Analysis-with-Pandas-2nd-edition/ch_04

In [ ]:
import numpy as np
import pandas as pd

fb = pd.read_csv('data/fb_2018.csv', index_col='date', parse_dates=True).assign(
    trading_volume=lambda x: pd.cut(x.volume, bins=3, labels=['low', 'med', 'high'])
)
fb.head()

In [ ]:
weather = pd.read_csv('data/weather_by_station.csv', index_col='date', parse_dates=True)
weather.head()

Before we dive into any calculations, let's make sure `pandas` won't put things in scientific notation. We will modify how floats are formatted for displaying. The format we will apply is `.2f`, which will provide the float with 2 digits after the decimal point:

In [ ]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

## Summarizing DataFrames
We learned about `agg()` in the [`2-dataframe_operations.ipynb`](./2-dataframe_operations.ipynb) notebook when we learned about window calculations; however, we can call this on the dataframe directly to aggregate its contents into a single series:

In [ ]:
fb.agg({
    'open': np.mean, 
    'high': np.max, 
    'low': np.min, 
    'close': np.mean, 
    'volume': np.sum
})

We can use this to find the total snowfall and precipitation recorded in Central Park in 2018:

In [ ]:
weather.query('station == "GHCND:USW00094728"')\
    .pivot(columns='datatype', values='value')[['SNOW', 'PRCP']]\
    .sum()

This is equivalent to passing `'sum'` to `agg()`:

In [ ]:
weather.query('station == "GHCND:USW00094728"')\
    .pivot(columns='datatype', values='value')[['SNOW', 'PRCP']]\
    .agg('sum')

Note that we aren't limited to providing a single aggregation per column. We can pass a list, and we will get a dataframe back instead of a series. Null values are placed where we don't have a calculation result to display:

In [ ]:
fb.agg({
    'open': 'mean',
    'high': ['min', 'max'],
    'low': ['min', 'max'],
    'close': 'mean'
})

## Using `groupby()`
Often we won't want to aggregate on the entire dataframe, but on groups within it. For this purpose, we can run `groupby()` before the aggregation. If we group by the `trading_volume` column, we will get a row for each of the values it takes on:

In [ ]:
fb.groupby('trading_volume').mean()

After we call `groupby()`, we can still select columns for aggregation:

In [ ]:
fb.groupby('trading_volume')['close'].agg(['min', 'max', 'mean'])

We can still provide a dictionary specifying the aggregations to perform, but passing a list for a column will result in a hierarchical index for the columns:

In [ ]:
fb_agg = fb.groupby('trading_volume').agg({
    'open': 'mean',
    'high': ['min', 'max'],
    'low': ['min', 'max'],
    'close': 'mean'
})
fb_agg

The hierarchical index in the columns looks like this:

In [ ]:
fb_agg.columns

Using a list comprehension, we can join the levels (in a tuple) with an `_` at each iteration: 

In [ ]:
fb_agg.columns = ['_'.join(col_agg) for col_agg in fb_agg.columns]
fb_agg.head()

We can group on values in the index if we tell `groupby()`, which `level` to use:

In [ ]:
weather.loc['2018-10'].query('datatype == "PRCP"')\
    .groupby(level=0).mean().head().squeeze()

We can also create a `Grouper` object, which can also roll up the datetimes in the index. Here, we find the quarterly total precipitation per station:

In [ ]:
weather.query('datatype == "PRCP"').groupby(
    ['station_name', pd.Grouper(freq='Q')]
).sum().unstack().sample(5, random_state=1)

Note that we can use `filter()` to exclude some groups from aggregation. Here, we only keep groups with names ending in "NY US" in the group's `name` attribute, which is the station name in this case:

In [ ]:
weather.groupby('station_name').filter( # station names with "NY US" in them
    lambda x: x.name.endswith('NY US')
).query('datatype == "SNOW"').groupby('station_name').sum().squeeze() # aggregate and make a series (squeeze)

Let's see which months have the most precipitation. First, we need to group by day and average the precipitation across the stations. Then we can group by month and sum the resulting precipitation. We use `nlargest()` to give the 5 months with the most precipitation:

In [ ]:
weather.query('datatype == "PRCP"')\
    .groupby(level=0).mean()\
    .groupby(pd.Grouper(freq='M')).sum().value.nlargest()

Perhaps the previous result was surprising. The saying goes "April showers bring May flowers"; yet April wasn't in the top 5 (neither was May for that matter). Snow will count towards precipitation, but that doesn't explain why summer months are higher than April. Let's look for days that accounted for a large percentage of the precipitation in a given month. 

In order to do so, we need to calculate the average daily precipitation across stations and then find the total per month. This will be the denominator. However, in order to divide the daily values by the total for their month, we will need a series of equal dimensions. This means we will need to use `transform()`:

In [ ]:
weather.query('datatype == "PRCP"')\
    .rename(dict(value='prcp'), axis=1)\
    .groupby(level=0).mean()\
    .groupby(pd.Grouper(freq='M'))\
    .transform(np.sum)['2018-01-28':'2018-02-03']

Notice how we have the same value repeated for each day in the month it belongs to. This will allow us to calculate the percentage of the monthly precipitation that occurred each day and then pull out the largest values:

In [ ]:
weather\
    .query('datatype == "PRCP"')\
    .rename(dict(value='prcp'), axis=1)\
    .groupby(level=0).mean()\
    .assign(
        total_prcp_in_month=lambda x: \
            x.groupby(pd.Grouper(freq='M')).transform(np.sum),
        pct_monthly_prcp=lambda x: \
            x.prcp.div(x.total_prcp_in_month)
    )\
    .nlargest(5, 'pct_monthly_prcp')

`transform()` can be used on dataframes as well. We can use it to easily standardize the data:

In [ ]:
fb[['open', 'high', 'low', 'close']]\
    .transform(lambda x: (x - x.mean()).div(x.std()))\
    .head()

## Pivot tables and crosstabs
We saw pivots in [`ch_03/4-reshaping_data.ipynb`](../ch_03/4-reshaping_data.ipynb); however, we weren't able to provide any aggregations. With `pivot_table()`, we get the mean by default. In its simplest form, we provide a column to place along the columns:

In [ ]:
fb.pivot_table(columns='trading_volume')

By placing the trading volume in the index, we get the transpose:

In [ ]:
fb.pivot_table(index='trading_volume')

With `pivot()`, we also weren't able to handle multi-level indices or indices with repeated values. For this reason we haven't been able to put the weather data in the wide format. The `pivot_table()` method solves this issue:

In [ ]:
weather.reset_index().pivot_table(
    index=['date', 'station', 'station_name'], 
    columns='datatype', 
    values='value',
    aggfunc='median'
).reset_index().tail()

We can use the `pd.crosstab()` function to create a frequency table. For example, if we want to see how many low-, medium-, and high-volume trading days Facebook stock had each month, we can use crosstab:

In [ ]:
pd.crosstab(
    index=fb.trading_volume,
    columns=fb.index.month,
    colnames=['month'] # name the columns index
)

We can normalize with the row or column totals with the `normalize` parameter. This shows percentage of the total:

In [ ]:
pd.crosstab(
    index=fb.trading_volume,
    columns=fb.index.month,
    colnames=['month'],
    normalize='columns'
)

If we want to perform a calculation other than counting the frequency, we can pass the column to run the calculation on to `values` and the function to use to `aggfunc`:

In [ ]:
pd.crosstab(
    index=fb.trading_volume,
    columns=fb.index.month,
    colnames=['month'],
    values=fb.close,
    aggfunc=np.mean
)

We can also get row and column subtotals with the `margins` parameter. Let's count the number of times each station recorded snow per month and include the subtotals:

In [ ]:
snow_data = weather.query('datatype == "SNOW"')
pd.crosstab(
    index=snow_data.station_name,
    columns=snow_data.index.month,
    colnames=['month'],
    values=snow_data.value,
    aggfunc=lambda x: (x > 0).sum(),
    margins=True, # show row and column subtotals
    margins_name='total observations of snow' # name the subtotals
)

<hr>
<div>
    <a href="./2-dataframe_operations.ipynb">
        <button>&#8592; Previous Notebook</button>
    </a>
    <a href="./4-time_series.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<hr>

# **4-4. time series**

# Working with Time Series Data

## About the Data
In this notebook, we will be working with 5 datasets:
- (CSV) Facebook's stock price daily throughout 2018 (obtained using the [`stock_analysis` package](https://github.com/stefmolin/stock-analysis)).
- (CSV) Facebook's OHLC stock data from May 20, 2019 - May 24, 2019 per minute from [Nasdaq.com](https://old.nasdaq.com/symbol/fb/interactive-chart).
- (CSV) melted stock data for Facebook from May 20, 2019 - May 24, 2019 per minute from [Nasdaq.com](https://old.nasdaq.com/symbol/fb/interactive-chart).
- (DB) stock opening prices by the minute for Apple from May 20, 2019 - May 24, 2019 altered to have seconds in the time from [Nasdaq.com](https://old.nasdaq.com/symbol/aapl/interactive-chart).
- (DB) stock opening prices by the minute for Facebook from May 20, 2019 - May 24, 2019 from [Nasdaq.com](https://old.nasdaq.com/symbol/fb/interactive-chart).

## Setup

In [ ]:
import numpy as np
import pandas as pd

fb = pd.read_csv('data/fb_2018.csv', index_col='date', parse_dates=True).assign(
    trading_volume=lambda x: pd.cut(x.volume, bins=3, labels=['low', 'med', 'high'])
)
fb.head()

## Time-based selection and filtering
Remember, when we have an index of type `DatetimeIndex`, we can use datetime slicing. We can provide a range of dates. We only get three days back because the stock market is closed on the weekends:

In [ ]:
fb['2018-10-11':'2018-10-15']

We can select ranges of months and quarters:

In [ ]:
fb.loc['2018-q1'].equals(fb['2018-01':'2018-03'])

The `first()` method will give us a specified length of time from the beginning of the time series. Here, we ask for a week. January 1, 2018 was a holiday—meaning the market was closed. It was also a Monday, so the week here is only four days:

In [ ]:
fb.first('1W')

The `last()` method will take from the end:

In [ ]:
fb.last('1W')

Suppose that we reindexed the Facebook stock data to include all dates for 2018. We would have null entries for January 1st:

In [ ]:
fb_reindexed = fb.reindex(pd.date_range('2018-01-01', '2018-12-31', freq='D'))
fb_reindexed.first('1D').isna().squeeze().all()

We can use `first_valid_index()` to give us the index of the first non-null entry in our data, which is the first day the market was open in Q1 2018:

In [ ]:
fb_reindexed.loc['2018-Q1'].first_valid_index()

Conversely, we can use `last_valid_index()` to get the last entry of non-null data. For Q1 2018, this is March 29th:

In [ ]:
fb_reindexed.loc['2018-Q1'].last_valid_index()

We can use `asof()` to find the last non-null data before the point we are looking for. If we ask for March 31st, we will get the data from the index we got from `fb_reindexed.loc['2018-Q1'].last_valid_index()`, which was March 29th. Note that this works regardless of whether we reindexed:

In [ ]:
fb_reindexed.asof('2018-03-31')

For the next few examples, we need datetimes, so we will read in the stock data per minute file:

In [ ]:
stock_data_per_minute = pd.read_csv(
    'data/fb_week_of_may_20_per_minute.csv', index_col='date', parse_dates=True, 
    date_parser=lambda x: pd.to_datetime(x, format='%Y-%m-%d %H-%M')
)

stock_data_per_minute.head()

We can use a `Grouper` object to roll up our data to the daily level along with `first` and `last`:

In [ ]:
stock_data_per_minute.groupby(pd.Grouper(freq='1D')).agg({
    'open': 'first',
    'high': 'max', 
    'low': 'min', 
    'close': 'last', 
    'volume': 'sum'
})

The `at_time()` method allows us to pull out all datetimes that match a certain time. Here, we can grab all the rows from the time the stock market opens (9:30 AM):

In [ ]:
stock_data_per_minute.at_time('9:30')

We can use `between_time()` to grab data for the last two minutes of trading daily:

In [ ]:
stock_data_per_minute.between_time('15:59', '16:00')

On average, are more shares traded within the first 30 minutes of trading or in the last 30 minutes? We can combine `between_time()` with `group_by()` and `filter()` from the [`3-aggregations.ipynb`](./3-aggregations.ipynb) notebook to answer this question. For the week in question, more are traded on average around opening time than closing time:

In [ ]:
shares_traded_in_first_30_min = stock_data_per_minute\
    .between_time('9:30', '10:00')\
    .groupby(pd.Grouper(freq='1D'))\
    .filter(lambda x: (x.volume > 0).all())\
    .volume.mean()

shares_traded_in_last_30_min = stock_data_per_minute\
    .between_time('15:30', '16:00')\
    .groupby(pd.Grouper(freq='1D'))\
    .filter(lambda x: (x.volume > 0).all())\
    .volume.mean()

shares_traded_in_first_30_min - shares_traded_in_last_30_min

In cases where time doesn't matter, we can normalize the times to midnight:

In [ ]:
pd.DataFrame(
    dict(before=stock_data_per_minute.index, after=stock_data_per_minute.index.normalize())
).head()

Note that we can also use `normalize()` on a `Series` object after accessing the `dt` attribute:

In [ ]:
stock_data_per_minute.index.to_series().dt.normalize().head()

## Shifting for lagged data
We can use `shift()` to create lagged data. By default, the shift will be one period. For example, we can use `shift()` to create a new column that indicates the previous day's closing price. From this new column, we can calculate the price change due to after-hours trading (after the close one day right up to the open the following day):

In [ ]:
fb.assign(
    prior_close=lambda x: x.close.shift(),
    after_hours_change_in_price=lambda x: x.open - x.prior_close,
    abs_change=lambda x: x.after_hours_change_in_price.abs()
).nlargest(5, 'abs_change')

If the goal is to to add/subtract time, we can use `pd.Timedelta` objects instead:

In [ ]:
pd.date_range('2018-01-01', freq='D', periods=5) + pd.Timedelta('9 hours 30 minutes')

## Differenced data
Using the `diff()` method is a quick way to calculate the difference between the data and a lagged version of itself. By default, it will yield the result of `data - data.shift()`:

In [ ]:
(
    fb.drop(columns='trading_volume') 
    - fb.drop(columns='trading_volume').shift()
).equals(
    fb.drop(columns='trading_volume').diff()
)

We can use this to see how Facebook stock changed day-over-day:

In [ ]:
fb.drop(columns='trading_volume').diff().head()

We can specify the number of periods, can be any positive or negative integer:

In [ ]:
fb.drop(columns='trading_volume').diff(-3).head()

## Resampling
Sometimes the data is at a granularity that isn't conducive to our analysis. Consider the case where we have data per minute for the full year of 2018. Let's see what happens if we try to plot this, and then look at the daily aggregation of this data.

*Plotting will be covered next chapter.*

In [ ]:
from visual_aids.misc_viz import resampling_example
resampling_example()

The plot on the left has so much data we can't see anything. However, when we aggregate to the daily totals, we see the data. We can alter the granularity of the data we are working with using resampling. Recall our minute-by-minute stock data:

In [ ]:
stock_data_per_minute.head()

We can resample this to get to a daily frequency:

In [ ]:
stock_data_per_minute.resample('1D').agg({
    'open': 'first',
    'high': 'max', 
    'low': 'min', 
    'close': 'last', 
    'volume': 'sum'
})

We can downsample to quarterly data:

In [ ]:
fb.resample('Q').mean()

We can also use `apply()`. Here, we show the quarterly change from start to end:

In [ ]:
fb.drop(columns='trading_volume').resample('Q').apply(
    lambda x: x.last('1D').values - x.first('1D').values
)

Consider the following melted stock data by the minute. We don't see the OHLC data directly:

In [ ]:
melted_stock_data = pd.read_csv('data/melted_stock_data.csv', index_col='date', parse_dates=True)
melted_stock_data.head()

We can use the `ohlc()` method after resampling to recover the OHLC columns:

In [ ]:
melted_stock_data.resample('1D').ohlc()['price']

Alternatively, we can upsample to increase the granularity. Note this will introduce `NaN` values:

In [ ]:
fb.resample('6H').asfreq().head()

There are many ways to handle these `NaN` values. We can forward-fill with `pad()`:

In [ ]:
fb.resample('6H').pad().head()

We can specify a specific value or a method with `fillna()`:

In [ ]:
fb.resample('6H').fillna('nearest').head()

We can use `asfreq()` and `assign()` to specify the action per column:

In [ ]:
fb.resample('6H').asfreq().assign(
    volume=lambda x: x.volume.fillna(0), # put 0 when market is closed
    close=lambda x: x.close.fillna(method='ffill'), # carry forward
    # take the closing price if these aren't available
    open=lambda x: x.open.combine_first(x.close),
    high=lambda x: x.high.combine_first(x.close),
    low=lambda x: x.low.combine_first(x.close)
).head()

## Merging
We saw merging examples the [`1-querying_and_merging.ipynb`](./1-querying_and_merging.ipynb) notebook. However, they all matched based on keys. With time series, it is possible that they are so granular that we never have the same time for multiple entries. Let's work with some stock data at different granularities:

In [ ]:
import sqlite3

with sqlite3.connect('data/stocks.db') as connection:
    fb_prices = pd.read_sql(
        'SELECT * FROM fb_prices', connection, 
        index_col='date', parse_dates=['date']
    )
    aapl_prices = pd.read_sql(
        'SELECT * FROM aapl_prices', connection, 
        index_col='date', parse_dates=['date']
    )

The Facebook prices are at the minute granularity:

In [ ]:
fb_prices.index.second.unique()

However, the Apple prices have information for the second:

In [ ]:
aapl_prices.index.second.unique()

We can perform an *as of* merge to try to line these up the best we can. We specify how to handle the mismatch with the `direction` and `tolerance` parameters. We will fill in with the `direction` of `nearest` and a `tolerance` of 30 seconds. This will place the Apple data with the minute that it is closest to, so 9:31:52 will go with 9:32 and 9:37:07 will go with 9:37. Since the times are on the index, we pass in `left_index` and `right_index`, as we did with `merge()` earlier this chapter: 

In [ ]:
pd.merge_asof(
    fb_prices, aapl_prices, 
    left_index=True, right_index=True, # datetimes are in the index
    # merge with nearest minute
    direction='nearest', tolerance=pd.Timedelta(30, unit='s')
).head()

If we don't want to lose the seconds information with the Apple data, we can use `pd.merge_ordered()` instead, which will interleave the two. Note this is an outer join by default (`how` parameter). The only catch here is that we need to reset the index in order to join on it:

In [ ]:
pd.merge_ordered(
    fb_prices.reset_index(), aapl_prices.reset_index()
).set_index('date').head()

We can pass a `fill_method` to handle `NaN` values:

In [ ]:
pd.merge_ordered(
    fb_prices.reset_index(), aapl_prices.reset_index(),
    fill_method='ffill'
).set_index('date').head()

Alternatively, we can use `fillna()`.

<hr>
<div style="overflow: hidden; margin-bottom: 10px;">
    <div style="float: left;">
         <a href="./3-aggregations.ipynb">
        <button>&#8592; Previous Notebook</button>
    </a>
    </div>
    <div style="float: right;">
        <a href="../../solutions/ch_04/solutions.ipynb">
            <button>Solutions</button>
        </a>
        <a href="../ch_05/1-introducing_matplotlib.ipynb">
            <button>Chapter 5 &#8594;</button>
        </a>
    </div>
</div>
<hr>

# **5-1. introduce matplotlib**

# Getting Started with Matplotlib

Pandas uses `matplotlib` to create visualizations. Therefore, before we learn how to plot with `pandas`, it's important to understand how `matplotlib` works at a high-level, which is the focus of this notebook.


## About the Data
In this notebook, we will be working with 2 datasets:
- Facebook's stock price throughout 2018 (obtained using the [`stock_analysis` package](https://github.com/stefmolin/stock-analysis))
- Earthquake data from September 18, 2018 - October 13, 2018 (obtained from the US Geological Survey (USGS) using the [USGS API](https://earthquake.usgs.gov/fdsnws/event/1/))

## Setup
We need to import `matplotlib.pyplot` for plotting.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

## Plotting lines

In [ ]:
fb = pd.read_csv(
    'data/fb_stock_prices_2018.csv', index_col='date', parse_dates=True
)

plt.plot(fb.index, fb.open)
plt.show()

Since we are working in a Jupyter notebook, we can use the magic command `%matplotlib inline` once and not have to call `plt.show()` for each plot.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd

fb = pd.read_csv(
    'data/fb_stock_prices_2018.csv', index_col='date', parse_dates=True
)
plt.plot(fb.index, fb.open)

## Scatter plots

We can pass in a string specifying the style of the plot. This is of the form `[marker][linestyle][color]`. For example, we can make a black dashed line with `'--k'` or a red scatter plot with `'or'`:

In [ ]:
plt.plot('high', 'low', 'or', data=fb.head(20))

Here are some examples of how you make a format string:

| Marker | Linestyle | Color | Format String | Result |
| :---: | :---: | :---: | :---: | --- |
| | `-` | `b` | `-b` | blue solid line|
| `.` |  | `k` | `.k` | black points|
|  | `--` | `r` | `--r` | red dashed line|
| `o` | `-` | `g` | `o-g` | green solid line with circles|
| | `:` | `m` | `:m` | magenta dotted line|
|`x` | `-.` | `c` | `x-.c` | cyan dot-dashed line with x's|
 
Note that we can also use format strings of the form `[color][marker][linestyle]`, but the parsing by `matplotlib` (in rare cases) might not be what we were aiming for. Consult the *Notes* section in the [documentation](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.plot.html) for the complete list of options.
## Histograms

In [ ]:
quakes = pd.read_csv('data/earthquakes.csv')
plt.hist(quakes.query('magType == "ml"').mag)

### Bin size matters
Notice how our assumptions of the distribution of the data can change based on the number of bins (look at the drop between the two highest peaks on the righthand plot):

In [ ]:
x = quakes.query('magType == "ml"').mag
fig, axes = plt.subplots(1, 2, figsize=(10, 3))
for ax, bins in zip(axes, [7, 35]):
    ax.hist(x, bins=bins)
    ax.set_title(f'bins param: {bins}')

## Plot components
### `Figure`
Top-level object that holds the other plot components.

In [ ]:
fig = plt.figure()

### `Axes`
Individual plots contained within the `Figure`.

## Creating subplots
Simply specify the number of rows and columns to create:

In [ ]:
fig, axes = plt.subplots(1, 2)

As an alternative to using `plt.subplots()` we can add `Axes` objects to the `Figure` object on our own. This allows for some more complex layouts, such as picture in picture:

In [ ]:
fig = plt.figure(figsize=(3, 3))
outside = fig.add_axes([0.1, 0.1, 0.9, 0.9])
inside = fig.add_axes([0.7, 0.7, 0.25, 0.25])

## Creating Plot Layouts with `gridspec`
We can create subplots with varying sizes as well:

In [ ]:
fig = plt.figure(figsize=(8, 8))
gs = fig.add_gridspec(3, 3)
top_left = fig.add_subplot(gs[0, 0])
mid_left = fig.add_subplot(gs[1, 0])
top_right = fig.add_subplot(gs[:2, 1:])
bottom = fig.add_subplot(gs[2,:])

## Saving plots
Use `plt.savefig()` to save the last created plot. To save a specific `Figure` object, use its `savefig()` method.

In [ ]:
fig.savefig('empty.png')

## Cleaning up
It's important to close resources when we are done with them. We use `plt.close()` to do so. If we pass in nothing, it will close the last plot, but we can pass in the specific `Figure` object to close or say `'all'` to close all `Figure` objects that are open. Let's close all the `Figure` objects that are open with `plt.close()`:

In [ ]:
plt.close('all')

## Additional plotting options
### Specifying figure size
Just pass the `figsize` argument to `plt.figure()`. It's a tuple of `(width, height)`:

In [ ]:
fig = plt.figure(figsize=(10, 4))

This can be specified when creating subplots as well:

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

### `rcParams`
A small subset of all the available plot settings (shuffling to get a good variation of options):

In [ ]:
import random
import matplotlib as mpl

rcparams_list = list(mpl.rcParams.keys())
random.seed(20) # make this repeatable
random.shuffle(rcparams_list)
sorted(rcparams_list[:20])

We can check the current default `figsize` using `rcParams`:

In [ ]:
mpl.rcParams['figure.figsize']

We can also update this value to change the default (until the kernel is restarted):

In [ ]:
mpl.rcParams['figure.figsize'] = (300, 10)
mpl.rcParams['figure.figsize']

Use `rcdefaults()` to restore the defaults. Note this is slightly different than before because running `%matplotlib inline` sets a different value for `figsize` ([see more](https://github.com/ipython/ipykernel/blob/master/ipykernel/pylab/config.py#L42-L56)). After we reset, we are going back to the default value of `figsize` before that import:

In [ ]:
mpl.rcdefaults()
mpl.rcParams['figure.figsize']

This can also be done via `pyplot`:

In [ ]:
plt.rc('figure', figsize=(20, 20)) # change `figsize` default to (20, 20)
plt.rcdefaults() # reset the default

<hr>
<div>
    <a href="../ch_04/4-time_series.ipynb">
        <button>&#8592; Chapter 4</button>
    </a>
    <a href="./2-plotting_with_pandas.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<hr>

# **5-2.plotting with pandas**

# Plotting with Pandas
The `plot()` method is available on `Series` and `DataFrame` objects. Many of the parameters get passed down to `matplotlib`. The `kind` argument let's us vary the plot type. Here are some commonly used parameters:

| Parameter | Purpose | Data Type |
| --- | --- | --- |
| `kind` | Determines the plot type | String |
| `x`/`y` | Column(s) to plot on the *x*-axis/*y*-axis | String or list |
| `ax` | Draws the plot on the `Axes` object provided | `Axes` |
| `subplots` | Determines whether to make subplots | Boolean |
| `layout` | Specifies how to arrange the subplots | Tuple of `(rows, columns)` |
| `figsize` | Size to make the `Figure` object | Tuple of `(width, height)` | 
| `title` | The title of the plot or subplots | String for the plot title or a list of strings for subplot titles |
| `legend` | Determines whether to show the legend | Boolean |
| `label` | What to call an item in the legend | String if a single column is being plotted; otherwise, a list of strings |
| `style` | `matplotlib` style strings for each item being plotted | String if a single column is being plotted; otherwise, a list of strings |
| `color` | The color to plot the item in | String or red, green, blue tuple if a single column is being plotted; otherwise, a list |
| `colormap` | The colormap to use | String or `matplotlib` colormap object |
| `logx`/`logy`/`loglog` | Determines whether to use a logarithmic scale for the *x*-axis, *y*-axis, or both | Boolean |
| `xticks`/`yticks` | Determines where to draw the ticks on the *x*-axis/*y*-axis | List of values |
| `xlim`/`ylim` | The axis limits for the *x*-axis/*y*-axis | Tuple of the form `(min, max)` |
| `rot` | The angle to write the tick labels at | Integer |
| `sharex`/`sharey` | Determines whether to have subplots share the *x*-axis/*y*-axis | Boolean |
| `fontsize` | Controls the size of the tick labels | Integer |
| `grid` | Turns on/off the grid lines | Boolean |

## About the Data
In this notebook, we will be working with 3 datasets:
- Facebook's stock price throughout 2018 (obtained using the [`stock_analysis` package](https://github.com/stefmolin/stock-analysis))
- Earthquake data from September 18, 2018 - October 13, 2018 (obtained from the US Geological Survey (USGS) using the [USGS API](https://earthquake.usgs.gov/fdsnws/event/1/))
- European Centre for Disease Prevention and Control's (ECDC) [daily number of new reported cases of COVID-19 by country worldwide dataset](https://www.ecdc.europa.eu/en/publications-data/download-todays-data-geographic-distribution-covid-19-cases-worldwide) collected on September 19, 2020 via [this link](https://opendata.ecdc.europa.eu/covid19/casedistribution/csv)

## Setup

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

fb = pd.read_csv(
    'data/fb_stock_prices_2018.csv', index_col='date', parse_dates=True
)
quakes = pd.read_csv('data/earthquakes.csv')
covid = pd.read_csv('data/covid19_cases.csv').assign(
    date=lambda x: pd.to_datetime(x.dateRep, format='%d/%m/%Y')
).set_index('date').replace(
    'United_States_of_America', 'USA'
).sort_index()['2020-01-18':'2020-09-18']

## Evolution over time
Line plots help us see how a variable changes over time. They are the default for the `kind` argument, but we can pass `kind='line'` to be explicit in our intent:

In [ ]:
fb.plot(
    kind='line',
    y='open',
    figsize=(10, 5),
    style='-b',
    legend=False,
    title='Evolution of Facebook Open Price'
)

We provided the `style` argument in the previous example; however, we can use the `color` and `linestyle` arguments to get the same result:

In [ ]:
fb.plot(
    kind='line',
    y='open',
    figsize=(10, 5),
    color='blue',
    linestyle='solid',
    legend=False,
    title='Evolution of Facebook Open Price'
)

We can also plot many lines at once by simply passing a list of the columns to plot:

In [ ]:
fb.first('1W').plot(
    y=['open', 'high', 'low', 'close'],
    style=['o-b', '--r', ':k', '.-g'],
    title='Facebook OHLC Prices during 1st Week of Trading 2018'
).autoscale()

### Creating subplots
When plotting with `pandas`, creating subplots is simply a matter of passing `subplots=True` to the `plot()` method, and (optionally) specifying the `layout` in a tuple of `(rows, columns)`:

In [ ]:
fb.plot(
    kind='line',
    subplots=True,
    layout=(3, 2),
    figsize=(15, 10),
    title='Facebook Stock 2018'
)

Note that we didn't provide a specific column to plot and `pandas` plotted all of them for us.

Sometimes we want to make subplots that each have a few variables in them for comparison. This can be achieved using the `ax` parameter. To illustrate this, let's take a look at daily new COVID-19 cases in China, Spain, Italy, the USA, Brazil, and India:

In [ ]:
new_cases_rolling_average = covid.pivot_table(
    index=covid.index, 
    columns='countriesAndTerritories', 
    values='cases'
).rolling(7).mean()

Since there is a lot of fluctuation in these values, we will plot the 7-day moving average of new cases using the `rolling()` method (discussed in chapter 4). Rather than create a separate plot for each country (which makes it harder to compare) or plot them all together (which will make it difficult to see the smaller values), we will plot countries that have had a similar number of cases in the same subplot:

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

new_cases_rolling_average[['China']].plot(ax=axes[0], style='-.c')
new_cases_rolling_average[['Italy', 'Spain']].plot(
    ax=axes[1], style=['-', '--'], 
    title='7-day rolling average of new COVID-19 cases\n(source: ECDC)'
)
new_cases_rolling_average[['Brazil', 'India', 'USA']]\
    .plot(ax=axes[2], style=['--', ':', '-'])

*NOTE: we specified the line styles here so that the lines can be distinguished in the text as a black and white image.*

In the previous figure, we were able to compare countries with similar levels of new COVID-19 cases, but we couldn't compare all of them in the same plot due to scale. One way around this is to use an **area plot**, which makes it possible for us to visualize the overall 7-day rolling average of new COVID-19 cases and at the same time how much each country is contributing to the total. In the interest of readability, we will group Italy and Spain together and create another category for countries other than the USA, Brazil, and India. The combined height of the plot areas is the overall value, and the height of given shaded region is the value for the individual country.

In [ ]:
cols = [
    col for col in new_cases_rolling_average.columns 
    if col not in ['USA', 'Brazil', 'India', 'Italy & Spain']
]
new_cases_rolling_average.assign(
    **{'Italy & Spain': lambda x: x.Italy + x.Spain}
).sort_index(axis=1).assign(
    Other=lambda x: x[cols].sum(axis=1)
).drop(columns=cols).plot(
    kind='area', figsize=(15, 5), 
    title='7-day rolling average of new COVID-19 cases\n(source: ECDC)'
)

Another way to visualize evolution over time is to look at the cumulative sum over time. Let's plot the cumulative number of COVID-19 cases in China, Spain, Italy, the USA, Brazil, and India, using `ax` to create subplots as we did in the previous example.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 3))

cumulative_covid_cases = covid.groupby(
    ['countriesAndTerritories', pd.Grouper(freq='1D')]
).cases.sum().unstack(0).apply('cumsum')

cumulative_covid_cases[['China']].plot(ax=axes[0], style='-.c')
cumulative_covid_cases[['Italy', 'Spain']].plot(
    ax=axes[1], style=['-', '--'], 
    title='Cumulative COVID-19 Cases\n(source: ECDC)'
)
cumulative_covid_cases[['Brazil', 'India', 'USA']]\
    .plot(ax=axes[2], style=['--', ':', '-'])

*NOTE: we specified the line styles here so that the lines can be distinguished in the text as a black and white image.*

## Visualizing relationships between variables
### Scatter plots
We make scatter plots to help visualize the relationship between two variables. Creating scatter plots requires we pass in `kind='scatter'` along with a column for the x-axis and a column for the y-axis:

In [ ]:
fb.assign(
    max_abs_change=fb.high - fb.low
).plot(
    kind='scatter', x='volume', y='max_abs_change',
    title='Facebook Daily High - Low vs. Volume Traded'
)

The relationship doesn't seem to be linear, but we can try a log transform on the x-axis since the scales of the axes are very different. With `pandas`, we simply pass in `logx=True`:

In [ ]:
fb.assign(
    max_abs_change=fb.high - fb.low
).plot(
    kind='scatter', x='volume', y='max_abs_change',
    title='Facebook Daily High - Low vs. log(Volume Traded)', 
    logx=True
)

With `matplotlib`, we could use `plt.xscale('log')` to do the same thing.

### Adding Transparency to Plots with `alpha`
Sometimes our plots have many overlapping values, but this can be impossible to see. This can be addressed by increasing the transparency of what we are plotting using the `alpha` parameter. It is a float in the range [0, 1] where 0 is completely transparent and 1 is completely opaque. By default this is 1, so let's put in a lower value and re-plot the scatter plot:

In [ ]:
fb.assign(
    max_abs_change=fb.high - fb.low
).plot(
    kind='scatter', x='volume', y='max_abs_change',
    title='Facebook Daily High - Low vs. log(Volume Traded)',
    logx=True, alpha=0.25
)

### Hexbins
In the previous example, we can start to see the overlaps, but it is still difficult. Hexbins are another plot type that divide up the plot into hexagons, which are shaded according to the density of points there. With `pandas`, this is the `hexbin` value for the `kind` argument. It may also be necessary to tweak the `gridsize`, which determines the number of hexagons along the y-axis:

In [ ]:
fb.assign(
    log_volume=np.log(fb.volume),
    max_abs_change=fb.high - fb.low
).plot(
    kind='hexbin',
    x='log_volume',
    y='max_abs_change',
    title='Facebook Daily High - Low vs. log(Volume Traded)',
    colormap='gray_r',
    gridsize=20, 
    sharex=False # we have to pass this to see the x-axis
)

### Visualizing Correlations with Heatmaps
Pandas doesn't offer heatmaps; however, if we are able to get our data into a matrix, we can use `matshow()` from matplotlib:

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))

# calculate the correlation matrix
fb_corr = fb.assign(
    log_volume=np.log(fb.volume),
    max_abs_change=fb.high - fb.low
).corr()

# create the heatmap and colorbar
im = ax.matshow(fb_corr, cmap='seismic')
im.set_clim(-1, 1)
fig.colorbar(im)

# label the ticks with the column names
labels = [col.lower() for col in fb_corr.columns]
ax.set_xticks(ax.get_xticks()[1:-1]) # to handle bug in matplotlib
ax.set_xticklabels(labels, rotation=45)
ax.set_yticks(ax.get_yticks()[1:-1]) # to handle bug in matplotlib
ax.set_yticklabels(labels)

# include the value of the correlation coefficient in the boxes
for (i, j), coef in np.ndenumerate(fb_corr):
    ax.text(
        i, j, fr'$\rho$ = {coef:.2f}', # raw (r), format (f) string
        ha='center', va='center', 
        color='white', fontsize=14
    )

Accessing the values in the correlation matrix can be done with `loc[]`:

In [ ]:
fb_corr.loc['max_abs_change', ['volume', 'log_volume']]

## Visualizing distributions
### Histograms
With the `pandas`, making histograms is as easy as passing `kind='hist'` to the `plot()` method:

In [ ]:
fb.volume.plot(
    kind='hist', 
    title='Histogram of Daily Volume Traded in Facebook Stock'
)
plt.xlabel('Volume traded') # label the x-axis (discussed in chapter 6)

We can overlap histograms to compare distributions provided we use the `alpha` parameter. For example, let's compare the usage and magnitude of the various measurement techniques (the `magType` column) in the data:

In [ ]:
fig, axes = plt.subplots(figsize=(8, 5))

for magtype in quakes.magType.unique():
    data = quakes.query(f'magType == "{magtype}"').mag
    if not data.empty:
        data.plot(
            kind='hist', ax=axes, alpha=0.4, 
            label=magtype, legend=True,
            title='Comparing histograms of earthquake magnitude by magType'
        )

plt.xlabel('magnitude') # label the x-axis (discussed in chapter 6)

### Kernel Density Estimation (KDE)
We can pass `kind='kde'` for an estimate of the probability density function (PDF), which tells us the probability of getting a particular value:

In [ ]:
fb.high.plot(
    kind='kde', 
    title='KDE of Daily High Price for Facebook Stock'
)
plt.xlabel('Price ($)') # label the x-axis (discussed in chapter 6)

### Adding to the result of `plot()`
The `plot()` method returns an `Axes` object. We can store this for additional customization of the plot, or we can pass this into another call to `plot()` as the `ax` argument to add to the original plot. 

It can often be helpful to view the KDE superimposed on top of the histogram, which can be achieved with this strategy:

In [ ]:
ax = fb.high.plot(kind='hist', density=True, alpha=0.5)
fb.high.plot(
    ax=ax, kind='kde', color='blue', 
    title='Distribution of Facebook Stock\'s Daily High Price in 2018'
)
plt.xlabel('Price ($)') # label the x-axis (discussed in chapter 6)

### Plotting the ECDF
In some cases, we are more interested in the probability of getting less than or equal to that value (or greater than or equal), which we can see with the **cumulative disribution function (CDF)**. Using the `statsmodels` package, we can estimate the CDF giving us the **empirical cumulative distribution function (ECDF)**:

In [ ]:
from statsmodels.distributions.empirical_distribution import ECDF

ecdf = ECDF(quakes.query('magType == "ml"').mag)
plt.plot(ecdf.x, ecdf.y)

# axis labels (we will cover this in chapter 6)
plt.xlabel('mag') # add x-axis label 
plt.ylabel('cumulative probability') # add y-axis label

# add title (we will cover this in chapter 6)
plt.title('ECDF of earthquake magnitude with magType ml')

This ECDF tells us the probability of getting an earthquake with magnitude of 3 or less using the `ml` scale is 98%:

In [ ]:
from statsmodels.distributions.empirical_distribution import ECDF

ecdf = ECDF(quakes.query('magType == "ml"').mag)
plt.plot(ecdf.x, ecdf.y)

# formatting below will all be covered in chapter 6
# axis labels
plt.xlabel('mag') # add x-axis label 
plt.ylabel('cumulative probability') # add y-axis label

# add reference lines for interpreting the ECDF for mag <= 3 
plt.plot(
    [3, 3], [0, .98], '--k', 
    [-1.5, 3], [0.98, 0.98], '--k', alpha=0.4
)

# set axis ranges
plt.ylim(0, None)
plt.xlim(-1.25, None)

# add a title
plt.title('P(mag <= 3) = 98%')

### Box plots
To make box plots with `pandas`, we pass `kind='box'` to the `plot()` method:

In [ ]:
fb.iloc[:,:4].plot(kind='box', title='Facebook OHLC Prices Box Plot')
plt.ylabel('price ($)') # label the x-axis (discussed in chapter 6)

If we pass in `notch=True`, we get a notched box plot. The notch represents a 95% confidence interval around the median, which can be helpful when comparing differences. For an introduction to interpreting a notched box plot, see this [Google sites page](https://sites.google.com/site/davidsstatistics/home/notched-box-plots) and this [Towards Data Science article](https://towardsdatascience.com/understanding-boxplots-5e2df7bcbd51).

In [ ]:
fb.iloc[:,:4].plot(kind='box', title='Facebook OHLC Prices Box Plot', notch=True)
plt.ylabel('price ($)') # label the x-axis (discussed in chapter 6)

This can also be combined with a call to `groupby()`:

In [ ]:
fb.assign(
    volume_bin=pd.cut(fb.volume, 3, labels=['low', 'med', 'high'])
).groupby('volume_bin').boxplot(
    column=['open', 'high', 'low', 'close'],
    layout=(1, 3), figsize=(12, 3)
)
plt.suptitle('Facebook OHLC Box Plots by Volume Traded', y=1.1)

We can use this to see the distribution of magnitudes across the different measurement methods for earthquakes:

In [ ]:
quakes[['mag', 'magType']].groupby('magType').boxplot(
    figsize=(15, 8), subplots=False
)
plt.title('Earthquake Magnitude Box Plots by magType')
plt.ylabel('magnitude') # label the y-axis (discussed in chapter 6)

## Counts and frequencies
### Bar charts
Passing `kind='barh'` gives us horizontal bars while `kind='bar'` gives us vertical ones. Let's use horizontal bars to look at the top 15 places for earthquakes in our data:

In [ ]:
quakes.parsed_place.value_counts().iloc[14::-1,].plot(
    kind='barh', figsize=(10, 5),
    title='Top 15 Places for Earthquakes '
          '(September 18, 2018 - October 13, 2018)'
)
plt.xlabel('earthquakes') # label the x-axis (discussed in chapter 6)

We also have data on whether earthquakes were accompanied by tsunamis. Let's see what the top places for tsunamis are:

In [ ]:
quakes.groupby('parsed_place').tsunami.sum().sort_values().iloc[-10:,].plot(
    kind='barh', figsize=(10, 5), 
    title='Top 10 Places for Tsunamis '
          '(September 18, 2018 - October 13, 2018)'
)
plt.xlabel('tsunamis') # label the x-axis (discussed in chapter 6)

Seeing that Indonesia is the top place for tsunamis during the time period we are looking at, we may want to look how many earthquakes and tsunamis Indonesia gets on a daily basis. We could show this as a line plot or with bars; since we don't want to interpolate, we will use bars here:

In [ ]:
indonesia_quakes = quakes.query('parsed_place == "Indonesia"').assign(
    time=lambda x: pd.to_datetime(x.time, unit='ms'),
    earthquake=1
).set_index('time').resample('1D').sum()

# format the datetimes in the index for the x-axis
indonesia_quakes.index = indonesia_quakes.index.strftime('%b\n%d')

indonesia_quakes.plot(
    y=['earthquake', 'tsunami'], kind='bar', figsize=(15, 3), 
    rot=0, label=['earthquakes', 'tsunamis'], 
    title='Earthquakes and Tsunamis in Indonesia '
          '(September 18, 2018 - October 13, 2018)'
)

# label the axes (discussed in chapter 6)
plt.xlabel('date')
plt.ylabel('count')

### Grouped Bars

In [ ]:
quakes.groupby(['parsed_place', 'tsunami']).mag.count()\
    .unstack().apply(lambda x: x / x.sum(), axis=1)\
    .rename(columns={0: 'no', 1: 'yes'})\
    .sort_values('yes', ascending=False)[7::-1]\
    .plot.barh(
        title='Frequency of a tsunami accompanying an earthquake'
    )

# move legend to the right of the plot
plt.legend(title='tsunami?', bbox_to_anchor=(1, 0.65))

# label the axes (discussed in chapter 6)
plt.xlabel('percentage of earthquakes')
plt.ylabel('')

Using the `kind` arugment for vertical bars when the labels for each bar are shorter:

In [ ]:
quakes.magType.value_counts().plot(
    kind='bar', title='Earthquakes Recorded per magType', rot=0
)

# label the axes (discussed in chapter 6)
plt.xlabel('magType')
plt.ylabel('earthquakes')

### Stacked bars

In [ ]:
pivot = quakes.assign(
    mag_bin=lambda x: np.floor(x.mag)
).pivot_table(
    index='mag_bin', columns='magType', values='mag', aggfunc='count'
)
pivot.plot.bar(
    stacked=True, rot=0, ylabel='earthquakes', 
    title='Earthquakes by integer magnitude and magType'
)

#### Normalized stacked bars
Plot the percentages to be better able to see the different `magTypes`.

In [ ]:
normalized_pivot = pivot.fillna(0).apply(lambda x: x / x.sum(), axis=1)
ax = normalized_pivot.plot.bar(
    stacked=True, rot=0, figsize=(10, 5),
    title='Percentage of earthquakes by integer magnitude for each magType'
)
ax.legend(bbox_to_anchor=(1, 0.8)) # move legend to the right of the plot
plt.ylabel('percentage') # label the axes (discussed in chapter 6)

We can also create horizontal stacked bars and do so using `groupby()` and `unstack()`:

In [ ]:
quakes.groupby(['parsed_place', 'tsunami']).mag.count()\
    .unstack().apply(lambda x: x / x.sum(), axis=1)\
    .rename(columns={0: 'no', 1: 'yes'})\
    .sort_values('yes', ascending=False)[7::-1]\
    .plot.barh(
        title='Frequency of a tsunami accompanying an earthquake', 
        stacked=True
    )

# move legend to the right of the plot
plt.legend(title='tsunami?', bbox_to_anchor=(1, 0.65))

# label the axes (discussed in chapter 6)
plt.xlabel('percentage of earthquakes')
plt.ylabel('')

<hr>
<div>
    <a href="./1-introducing_matplotlib.ipynb">
        <button>&#8592; Previous Notebook</button>
    </a>
    <a href="./3-pandas_plotting_module.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<hr>

# **5-3. pandas plotting module**

# The `pandas.plotting` module
Pandas provides some extra plotting functions for some new plot types.

## About the Data
In this notebook, we will be working with Facebook's stock price throughout 2018 (obtained using the [`stock_analysis` package](https://github.com/stefmolin/stock-analysis)).

## Setup

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

fb = pd.read_csv(
    'data/fb_stock_prices_2018.csv', index_col='date', parse_dates=True
)

## Scatter matrix
Easily create scatter plots between all columns in the dataset:

In [ ]:
from pandas.plotting import scatter_matrix
scatter_matrix(fb, figsize=(10, 10))

Changing the diagonal from histograms to KDE:

In [ ]:
scatter_matrix(fb, figsize=(10, 10), diagonal='kde')

## Lag plot
Lag plots let us see how the variable correlates with past observations of itself. Random data has no pattern:

In [ ]:
from pandas.plotting import lag_plot
np.random.seed(0) # make this repeatable
lag_plot(pd.Series(np.random.random(size=200)))

Data with some level of correlation to itself (autocorrelation) may have patterns. Stock prices are highly autocorrelated:

In [ ]:
lag_plot(fb.close)

The default lag is 1, but we can alter this with the `lag` parameter. Let's look at a 5 day lag (a week of trading activity):

In [ ]:
lag_plot(fb.close, lag=5)

## Autocorrelation plots
We can use the autocorrelation plot to see if this relationship may be meaningful or is just noise. Random data will not have any significant autocorrelation (it stays within the bounds below):

In [ ]:
from pandas.plotting import autocorrelation_plot
np.random.seed(0) # make this repeatable
autocorrelation_plot(pd.Series(np.random.random(size=200)))

Stock data, on the other hand, does have significant autocorrelation:

In [ ]:
autocorrelation_plot(fb.close)

## Bootstrap plot
This plot helps us understand the uncertainty in our summary statistics:

In [ ]:
from pandas.plotting import bootstrap_plot
fig = bootstrap_plot(fb.volume, fig=plt.figure(figsize=(10, 6)))

<hr>

<div style="overflow: hidden; margin-bottom: 10px;">
    <div style="float: left;">
        <a href="./2-plotting_with_pandas.ipynb">
            <button>&#8592; Previous Notebook</button>
        </a>
    </div>
    <div style="float: right;">
        <a href="../../solutions/ch_05/solutions.ipynb">
            <button>Solutions</button>
        </a>
        <a href="../ch_06/1-introduction_to_seaborn.ipynb">
            <button>Chapter 6 &#8594;</button>
        </a>
    </div>
</div>
<hr>